In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast # para la transformacion de str a array
import requests
#import matplotlib.pyplot as plt
from PIL import Image
import re
import pandasql as sql # para trabajar con los datasets
from tqdm import tqdm # barra de progreso
from concurrent.futures import ProcessPoolExecutor # para paralelismo
from concurrent.futures import ThreadPoolExecutor # para paralelismo
from concurrent.futures import as_completed  # para paralelismo

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import CLIPProcessor, CLIPModel
import spacy # para metrica de similitud

/home/angel/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-19 20:52:54.078106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 20:52:54.148413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 20:52:54.171025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 20:52:54.294442: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is o

In [51]:
%pip install semantic-text-similarity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 9.1 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.2 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Metrica semantica texto 
from semantic_text_similarity.models import WebBertSimilarity #(1)

In [2]:
model_id = "llava-hf/llava-1.5-7b-hf"; #llava-hf/llava-1.5-7b-hf modelos de mas parametros. llava-1.5-13b-hf 

model= LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
).to(0);

processor = AutoProcessor.from_pretrained(model_id);

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Carga de dataset procesados

In [12]:
dataset_humano_cifar=unpickle('dataset_humano_cifar_250_v4_29_06_2024.pickle')
dataset_animal_cifar=unpickle('dataset_animal_cifar_250_v3_29_06_2024.pickle')
dataset_objeto_cifar=unpickle('dataset_objeto_cifar_250_v3_29_06_2024.pickle')
dataset_paisaje_cifar=unpickle('dataset_paisaje_cifar_250_v3_29_06_2024.pickle')

In [5]:
dataset_humano_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
58,b'black_woman_s_001310.png',None,98,14,"[165, 164, 139, 122, 113, 93, 74, 63, 67, 70, ...",woman,singular,Human
65,b'female_child_s_000292.png',None,35,14,"[220, 208, 213, 232, 239, 230, 163, 143, 150, ...",girl,singular,Human
77,b'man_s_000841.png',None,46,14,"[100, 117, 81, 92, 110, 152, 129, 107, 125, 11...",man,singular,Human
90,b'man_s_000737.png',None,46,14,"[97, 93, 94, 94, 88, 91, 91, 72, 57, 69, 71, 6...",man,singular,Human
92,b'girl_s_001964.png',None,35,14,"[61, 61, 62, 63, 63, 61, 60, 67, 81, 103, 128,...",girl,singular,Human
...,...,...,...,...,...,...,...,...
6149,b'abel_s_000450.png',None,46,14,"[213, 211, 212, 212, 212, 212, 212, 212, 212, ...",man,singular,Human
6159,b'woman_s_000104.png',None,98,14,"[121, 108, 95, 62, 47, 70, 55, 55, 95, 55, 33,...",woman,singular,Human
6216,b'young_girl_s_000399.png',None,35,14,"[229, 141, 123, 116, 146, 111, 93, 113, 42, 25...",girl,singular,Human
6223,b'infant_s_001111.png',None,2,14,"[85, 77, 75, 77, 71, 64, 99, 114, 95, 133, 158...",baby,singular,Human


In [5]:
dataset_animal_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
0,b'bos_taurus_s_000507.png',116,19,11,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",cattle,plural,Animal
1,b'stegosaurus_s_000125.png',114,29,15,"[255, 253, 253, 253, 253, 253, 253, 253, 253, ...",dinosaur,singular,Animal
4,b'cichlid_s_000031.png',110,1,1,"[43, 32, 87, 127, 55, 48, 51, 87, 121, 113, 13...",aquarium_fish,singular,Animal
9,b'rogue_elephant_s_000421.png',98,31,11,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",elephant,singular,Animal
17,b'wolf_pup_s_001323.png',111,97,8,"[167, 161, 170, 168, 163, 166, 165, 166, 167, ...",wolf,singular,Animal
...,...,...,...,...,...,...,...,...
904,b'winter_flounder_s_000509.png',None,32,1,"[234, 232, 232, 231, 230, 230, 230, 230, 230, ...",flatfish,singular,Animal
905,b'worm_snake_s_001265.png',None,78,15,"[60, 70, 71, 96, 137, 144, 63, 25, 8, 28, 116,...",snake,singular,Animal
906,b'stegosaurus_s_000069.png',None,29,15,"[71, 72, 80, 78, 76, 52, 35, 26, 16, 17, 16, 1...",dinosaur,singular,Animal
910,b'tiger_s_001191.png',None,88,8,"[241, 232, 244, 251, 250, 250, 248, 242, 245, ...",tiger,singular,Animal


In [6]:
dataset_objeto_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
2,b'mcintosh_s_000643.png',97,0,4,"[250, 248, 247, 248, 249, 249, 248, 248, 247, ...",apple,singular,Object
5,b'phone_s_002161.png',105,86,5,"[190, 191, 194, 191, 190, 192, 195, 197, 195, ...",telephone,singular,Object
7,b'beaker_s_000604.png',103,28,3,"[178, 175, 175, 175, 176, 176, 176, 176, 176, ...",cup,singular,Object
16,b'bike_s_000682.png',32,8,18,"[35, 27, 31, 77, 136, 168, 103, 101, 85, 63, 6...",bicycle,singular,Object
23,b'surveillance_system_s_000769.png',None,87,5,"[255, 252, 248, 243, 235, 224, 221, 222, 223, ...",television,singular,Object
...,...,...,...,...,...,...,...,...
1206,b'ballistic_missile_s_000772.png',None,69,19,"[199, 176, 169, 156, 141, 131, 124, 115, 111, ...",rocket,singular,Object
1209,b'mushroom_s_001277.png',None,51,4,"[160, 138, 134, 140, 133, 138, 140, 169, 157, ...",mushroom,singular,Object
1210,b'wing_chair_s_000220.png',None,20,6,"[246, 238, 227, 215, 211, 209, 210, 206, 188, ...",chair,singular,Object
1211,b'pickup_truck_s_000832.png',None,58,18,"[129, 124, 112, 115, 131, 133, 152, 165, 148, ...",pickup_truck,singular,Object


# Funcion de concatenacion

In [3]:
"Concatenacion preguntas"
def preguntar (pregunta):
    
    pregunta=f'USER: <image>\n"{pregunta}"\nASSISTANT:';
    
    return pregunta

# Preguntas

In [5]:
"PERSONAS"
sujeto_extraido='';

preguntas_max_humano=[
preguntar("what is the background of the image?"), 
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there humans?"),
preguntar("Are there animals?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} young or old?"),
preguntar(f"Are There {sujeto_extraido} male?"),
preguntar(f"Are There {sujeto_extraido} female?"),                               
preguntar("What genders are there?"),
preguntar("What traits could be differentiated from other gender?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"), 
preguntar("What physical characteristics could be differentiated by other gender?"),
preguntar(f"Which {sujeto_extraido} is the male?"),
preguntar(f"{sujeto_extraido} have similar colors?"), # condicional plural el insert se encarga
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicional plural el insert se encarga
preguntar(f"Do {sujeto_extraido} interact with each other?"), # condicional plural el insert se encarga
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic or body part to define and determine?"),
preguntar(f"If {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in skin color, clothing or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features or facial expressions of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} clothing to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")
          ]    
len(preguntas_max_humano)

58

In [14]:
"ANIMALES"
preguntas_max_animales=[
preguntar("what is the background of the image?"),
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there humans?"),
preguntar("Are there animals?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} young or old?"),
preguntar(f"Are There {sujeto_extraido} male?"),
preguntar(f"Are There {sujeto_extraido} famale?"),                      
preguntar("What genders are there?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
preguntar("What physical characteristics could be differentiated other gender?"),
preguntar(f"Which {sujeto_extraido} is the male?"),
preguntar(f"{sujeto_extraido} have similar colors?"), # condicionada plural 24
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicionada plural 25
preguntar(f"Do {sujeto_extraido} interact with each other?"), # condicionada plural 26
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic or body part to define and determine?"),
preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"), ##---eficiencia
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in skin color or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features or facial expressions of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} skin color to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")]

len(preguntas_max_animales)

57

In [15]:
"OBJETOS"
preguntas_max_objetos=[
preguntar("what is the background of the image?"), 
preguntar("Is it rainy or sunny?"),
preguntar("Is it a closed or open place?"),
preguntar("Is the image bright?"),
preguntar("Are there elements in the image that can make visual noise?"),
preguntar("Are there a lot of different entities?"),
preguntar("Where is the image taken from?"),
preguntar("Are there objects?"),
preguntar(f"How many are {sujeto_extraido} there?"),
preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
preguntar(f"In what position are {sujeto_extraido} located?"),
preguntar("which actions are shown in the image"),
preguntar("Could you indicate the reasons why that action is performed?"),
preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
preguntar("Do you know what are the benefits or consequences of that action?"),
preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
preguntar(f"Are {sujeto_extraido} new or old?"),
preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
preguntar(f"What physical characteristics could be differentiated other {sujeto_extraido}?"),   
preguntar(f"{sujeto_extraido} have similar colors?"), # condicionada
preguntar(f"Are {sujeto_extraido} far away from each other?"), # condicionada
preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
preguntar("What is the most complicated characteristic to define and determine?"),
preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
preguntar(f"With which other {sujeto_extraido} would the prediction be most likely to give a better result?"),
preguntar(f"If there are subtle changes in color or posture, would the result of the prediction that they are {sujeto_extraido} be the same?"),
preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
preguntar("Could you describe what is happening in the image?"),
preguntar("Could the situation you predicted be used in other areas?"),
preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
preguntar("How could the situation you have predicted above be better?"),
preguntar("What would an image similar to the situation found in this image look like?"),
preguntar("Can you describe a similar counterexample to the situation you have predicted above?"),
preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
preguntar("Could you in simpler words describe their function that you have predicted before?"),
preguntar("What could stand out from the situation described above and be useful in similar situations?"),
preguntar("Can you give essential concepts that are relevant to the situation described?"),
preguntar("What aspects of the image have been detected that may attract attention?"),
preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
preguntar(f"Do you focus on features of the {sujeto_extraido} to result in saying they are {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} color to result in the {sujeto_extraido} being {sujeto_extraido}?"),
preguntar(f"Do you focus on the {sujeto_extraido} shape to result in the result that they are {sujeto_extraido}?")]

len(preguntas_max_objetos)

51

In [13]:
aqui esta como se debe colocar la funcion apra que coja el sujeto extraido

SyntaxError: invalid syntax (4159629101.py, line 1)

In [9]:
"PAISAJES"
def preguntas_max_paisajes_funcion(sujeto_extraido):
    preguntas_max_paisajes =[
    preguntar("what is the background of the image?"), 
    preguntar("Is it rainy or sunny?"),
    preguntar("Is it a closed or open place?"),
    preguntar("Is the image bright?"),
    preguntar("Are there elements in the image that can make visual noise?"),
    preguntar("Are there a lot of different entities?"),
    preguntar("Where is the image taken from?"),
    preguntar("Are there humans?"),
    preguntar("Are there animals?"),
    preguntar(f"How many are {sujeto_extraido} there?"),
    preguntar(f"Do {sujeto_extraido} have any elements that characterize them? Which ones?"),
    preguntar(f"In what position are {sujeto_extraido} located?"),
    preguntar("which actions are shown in the image"),
    preguntar("Could you indicate the reasons why that action is performed?"),
    preguntar(f"would the {sujeto_extraido} be able to perform any other action?"),
    preguntar("Do you know what are the benefits or consequences of that action?"),
    preguntar(f"How could you associate the {sujeto_extraido} with their environment to describe the action?"),
    preguntar(f"Are they {sujeto_extraido} young or old?"),
    preguntar(f"What features could be differentiated from other types of {sujeto_extraido}?"),
    preguntar(f"What physical characteristics could be differentiated from the opposite {sujeto_extraido}?"),
    preguntar(f"{sujeto_extraido} have similar colors?"), # coindicionada
    preguntar(f"Have you been able to identify how many {sujeto_extraido} there are?"),
    preguntar(f"If the {sujeto_extraido} change positions could you have another prediction of the situation described?"),
    preguntar(f"If the {sujeto_extraido} were doing another action, would it be another type of {sujeto_extraido}?"),
    preguntar(f"Are there elements in the image that could be noisy enough to intervene in the prediction that they are {sujeto_extraido}?"),
    preguntar(f"What physical traits are considered for the prediction result to be {sujeto_extraido}?"),
    preguntar(f"Why do you think identifying that they are {sujeto_extraido} in the image is easy?"),
    preguntar(f"What makes you think the {sujeto_extraido} are actually {sujeto_extraido} and not something else?"),
    preguntar(f"With which other kind of {sujeto_extraido} would the prediction be most likely to give a better result?"),
    preguntar(f"If there are subtle changes in color or position, would the result of the prediction that they are {sujeto_extraido} be the same?"),
    preguntar(f"Is there anything that might create doubt or uncertainty for you to say they are {sujeto_extraido}?"),
    preguntar("Could you describe what is happening in the image?"),
    preguntar("Could the situation you predicted be used in other areas?"),
    preguntar("Could the situation you predicted have some example of counterfactual situation that could be useful?"),
    preguntar("How could the situation you have predicted above be better?"),
    preguntar("What would an image similar to the situation found in this image look like?"),
    preguntar("Can you describe a similar counterfactual to the situation you have predicted above?"),
    preguntar(f"What is the task or purpose of {sujeto_extraido}?"),
    preguntar("Could you in simpler words describe their function that you have predicted before?"),
    preguntar("What could stand out from the situation described above and be useful in similar situations?"),
    preguntar("Can you give essential concepts that are relevant to the situation described?"),
    preguntar("What aspects of the image have been detected that may attract attention?"),
    preguntar(f"Why has it been obtained that the prediction is that they are {sujeto_extraido}?"),
    preguntar(f"How did you get the result that they are {sujeto_extraido}?"),
    preguntar(f"What do you focus on to get the prediction that they are {sujeto_extraido}?"),
    preguntar(f"What do you focus on {sujeto_extraido} to result in or say that they are {sujeto_extraido}?"),
    preguntar(f"Do you focus on parts of the {sujeto_extraido} bodies to infer that they are {sujeto_extraido}?"),
    preguntar(f"Do you focus on the {sujeto_extraido} body shape to result in the result that they are {sujeto_extraido}?")]

    return preguntas_max_paisajes

len(preguntas_max_paisajes)

48

In [ ]:
aqui esta lo que se debe sustituir

In [11]:
#prueba para probar lo del sujeto
def predit_prueba():

    sujeto_extraido="vaca"

    preguntas_por_hacer= preguntas_max_paisajes_funcion(sujeto_extraido)

    for preguntas in preguntas_por_hacer:
        print(preguntas)

In [12]:
predit_prueba()

USER: <image>
"what is the background of the image?"
ASSISTANT:
USER: <image>
"Is it rainy or sunny?"
ASSISTANT:
USER: <image>
"Is it a closed or open place?"
ASSISTANT:
USER: <image>
"Is the image bright?"
ASSISTANT:
USER: <image>
"Are there elements in the image that can make visual noise?"
ASSISTANT:
USER: <image>
"Are there a lot of different entities?"
ASSISTANT:
USER: <image>
"Where is the image taken from?"
ASSISTANT:
USER: <image>
"Are there humans?"
ASSISTANT:
USER: <image>
"Are there animals?"
ASSISTANT:
USER: <image>
"How many are vaca there?"
ASSISTANT:
USER: <image>
"Do vaca have any elements that characterize them? Which ones?"
ASSISTANT:
USER: <image>
"In what position are vaca located?"
ASSISTANT:
USER: <image>
"which actions are shown in the image"
ASSISTANT:
USER: <image>
"Could you indicate the reasons why that action is performed?"
ASSISTANT:
USER: <image>
"would the vaca be able to perform any other action?"
ASSISTANT:
USER: <image>
"Do you know what are the benefi

# funcion para aplicar pregunta

In [17]:
"Necesario para sacar el sujeto de la descripcion:"
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


"Para sacar el sujeto de la respues de descripcion"
def sujeto(respuesta_pregunta_descripcion):
    
    "Para sacar sujeto de description:"
    # Colocar la variable dentro del contexto raw string
    contexto = r"""{}""".format(respuesta_pregunta_descripcion)


    result = question_answerer(question="Who is being talked about?", context=contexto)
    sujeto= result['answer']
    
    print(sujeto)
    
    return sujeto


"Para sacar la cantiadd del sujeto que se encuentra"
def cantidad(respuesta_pregunta_descripcion):
    "Para sacar la cantidad:"
    # coloca la variable dentro del contexto raw string
    contexto = r"""{}""".format(respuesta_pregunta_descripcion)


    result = question_answerer(question="What numerical quantity of main entity is being discussed?", context=contexto) # aqui poner preguntar por el numero en la frasde
    cantidad= result['answer']
    
    print(cantidad)
    
    return cantidad



"Respuestas"
def predit(grupo,indice): # grupo 1 humano, grupo 2 animal, grupo 3 objeto, grupo 4 paisaje
        


    if grupo ==1:
        preguntas_hacer=preguntas_max_humano
        dataset=dataset_humano_cifar
    elif grupo ==2:
        preguntas_hacer=preguntas_max_animales
        dataset=dataset_animal_cifar
    elif grupo==3:
        preguntas_hacer=preguntas_max_objetos
        dataset=dataset_objeto_cifar
    elif grupo==4:
        preguntas_hacer=preguntas_max_paisajes
        dataset=dataset_paisaje_cifar
    else:
        pass
        

    # Extraer la imagen en formato numpy
    image_numpy= dataset[b'data'][indice]
    
     #dimensionar correctamente
    image_numpy= image_numpy.reshape(3, 32, 32).transpose(1,2,0)
    
    # pasando  de numpy a PIL
    raw_image =Image.fromarray(np.uint8(image_numpy), 'RGB')
    
    
    #sujeto extraido
    sujeto_extraido=dataset["etiqueta"][indice]
    
    #categoria
    categoria = dataset["categoria"][indice]
    
    #singularidad
    singularidad= dataset["singularidad"][indice]
    
    """
    if singularidad =='plural':
        preguntas_hacer.insert(24,"{sujeto_extraido} have similar colors?")
        preguntas_hacer.insert(25,"Are {sujeto_extraido} far away from each other?")
        preguntas_hacer.insert(26,"Do {sujeto_extraido} interact with each other?")
    else:
        preguntas_hacer.pop(24)
        preguntas_hacer.pop(24)
        preguntas_hacer.pop(24)
    """
    
    
    # Mostrar la imagen JPEG
    #plt.imshow(image_numpy, interpolation='bilinear')
    #plt.axis('off')
    #plt.show()

    #para guardar las respuestas
    #preguntas=[];
    respuestas=[];
    
    
    
    for prompt in preguntas_hacer:
        # input
        inputs = processor(prompt, raw_image, return_tensors='pt', max_length=512, padding='longest', truncation=True).to(model.device, torch.float16)        
        # output
        #output = model.generate(**inputs, max_new_tokens=200, num_beams=4, early_stopping=True)
        output = model.generate(**inputs, max_new_tokens=250, do_sample=False)

        #auxiliar para extraer preguntas y respuestas
        entrada_texto = processor.decode(output[0][2:], skip_special_tokens=True)

        inicio_indice= entrada_texto.find("ER:") + 4;
        final_indice = entrada_texto.find("\nASSISTANT");
        pregunta= entrada_texto[inicio_indice:final_indice];

        inicio_indice = entrada_texto.find("\nASSISTANT:") + len("\nASSISTANT:");
        respuesta = entrada_texto[final_indice+12:];

        respuestas.append(respuesta)
    
    del inputs, output, raw_image#, preguntas ya las preguntas no sirven
    torch.cuda.empty_cache()
    return respuestas

In [18]:
# creacion de dataset de preguntas y respuestas
dataset_humano_respuestas=pd.DataFrame(columns=preguntas_max_humano)
dataset_animales_respuestas=pd.DataFrame(columns=preguntas_max_animales)
dataset_objetos_respuestas=pd.DataFrame(columns=preguntas_max_objetos)
dataset_paisajes_respuestas=pd.DataFrame(columns=preguntas_max_paisajes)

In [19]:
dataset_humano_cifar

,b'filenames',b'batch_label',b'fine_labels',b'coarse_labels',b'data',etiqueta,singularidad,categoria
58,b'black_woman_s_001310.png',None,98,14,"[165, 164, 139, 122, 113, 93, 74, 63, 67, 70, ...",woman,singular,Human
65,b'female_child_s_000292.png',None,35,14,"[220, 208, 213, 232, 239, 230, 163, 143, 150, ...",girl,singular,Human
77,b'man_s_000841.png',None,46,14,"[100, 117, 81, 92, 110, 152, 129, 107, 125, 11...",man,singular,Human
90,b'man_s_000737.png',None,46,14,"[97, 93, 94, 94, 88, 91, 91, 72, 57, 69, 71, 6...",man,singular,Human
92,b'girl_s_001964.png',None,35,14,"[61, 61, 62, 63, 63, 61, 60, 67, 81, 103, 128,...",girl,singular,Human
...,...,...,...,...,...,...,...,...
6149,b'abel_s_000450.png',None,46,14,"[213, 211, 212, 212, 212, 212, 212, 212, 212, ...",man,singular,Human
6159,b'woman_s_000104.png',None,98,14,"[121, 108, 95, 62, 47, 70, 55, 55, 95, 55, 33,...",woman,singular,Human
6216,b'young_girl_s_000399.png',None,35,14,"[229, 141, 123, 116, 146, 111, 93, 113, 42, 25...",girl,singular,Human
6223,b'infant_s_001111.png',None,2,14,"[85, 77, 75, 77, 71, 64, 99, 114, 95, 133, 158...",baby,singular,Human


In [6]:
indices_dataset_humano_cifar=dataset_humano_cifar.index
indices_dataset_animal_cifar=dataset_animal_cifar.index
indices_dataset_objeto_cifar=dataset_objeto_cifar.index
indices_dataset_paisaje_cifar=dataset_paisaje_cifar.index

In [4]:
try:
    for x in tqdm(indices_dataset_humano_cifar[:50]):
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
except:
    print("error")

100%|██████████| 50/50 [4:03:08<00:00, 291.78s/it]  


In [13]:
dataset_humano_respuestas

,"USER: <image>\n""what is the background of the image?""\nASSISTANT:","USER: <image>\n""Is it rainy or sunny?""\nASSISTANT:","USER: <image>\n""Is it a closed or open place?""\nASSISTANT:","USER: <image>\n""Is the image bright?""\nASSISTANT:","USER: <image>\n""Are there elements in the image that can make visual noise?""\nASSISTANT:","USER: <image>\n""Are there a lot of different entities?""\nASSISTANT:","USER: <image>\n""Where is the image taken from?""\nASSISTANT:","USER: <image>\n""Are there humans?""\nASSISTANT:","USER: <image>\n""Are there animals?""\nASSISTANT:","USER: <image>\n""How many are there?""\nASSISTANT:",...,"USER: <image>\n""Can you give essential concepts that are relevant to the situation described?""\nASSISTANT:","USER: <image>\n""What aspects of the image have been detected that may attract attention?""\nASSISTANT:","USER: <image>\n""Why has it been obtained that the prediction is that they are ?""\nASSISTANT:","USER: <image>\n""How did you get the result that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to get the prediction that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to result in or say that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on parts of the bodies to infer that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on features or facial expressions of the to result in saying they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on the clothing to result in the being ?""\nASSISTANT:","USER: <image>\n""Do you focus on the body shape to result in the result that they are ?""\nASSISTANT:"
2106,The background of the image is a blue and purp...,Sunny,It is an open place.,"Yes, the image is bright, featuring a woman wi...","Yes, there are elements in the image that can ...","No, the image only features one woman with lon...",The image is taken from a close-up perspective...,Yes,No,"There is only one person in the image, a woman...",...,"In the image, a woman with long blonde hair is...","In the image, a woman with long blonde hair is...","The image features a woman with long hair, and...",The result that the woman in the image is a pa...,"To predict that the image features a woman, I ...","In the image, the focus is on the woman's face...","Yes, in the image, the focus is on the woman's...","Yes, the image focuses on the facial features ...","Yes, the focus on the clothing in the image, s...","Yes, the focus on the body shape and the woman..."
2131,The background of the image is blue.,It is sunny in the image.,"It is an open place, as the girl is smiling an...","Yes, the image is bright, as it is a close-up ...","Yes, there are elements in the image that can ...","No, the image only shows a single person, a yo...",The image is taken from a close-up perspective...,Yes,No,There are two people in the image.,...,"In the image, a young woman is smiling and loo...","In the image, a young woman with blonde hair i...",The image shows a young woman with blonde hair...,The result that the person in the image is a y...,To predict that the person in the image is a w...,"In the image, the focus is on the young girl's...","Yes, the image shows a close-up of a woman's f...","In the image, the girl is smiling and has a br...","Yes, the image focuses on the girl's clothing,...","Yes, the image shows a young woman with a smil..."
2179,"The background of the image is blurry, which s...",It is sunny in the image.,"It is an open place, as the man is standing in...","Yes, the image is bright, as it is described a...","Yes, there are elements in the image that can ...","Yes, there are many different entities in the ...",The image is taken from the perspective of the...,"Yes, there are humans in the image.",No,"There are several people in the image, includi...",...,"In the image, a man is holding a camera and lo...","In the image, a man is holding a camera, which...","The image shows a man holding a camera, with a...",The result that the people in the image are 

In [1]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte1.pkl')

# 2 tanda dataset humano

In [2]:
try:
    for x in tqdm(indices_dataset_humano_cifar[50:100]):
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
except:
    print("error")

100%|██████████| 50/50 [3:59:35<00:00, 287.51s/it]  


In [ ]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte2.pkl')

In [6]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [22]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [23]:
!#export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
# Desactivar la paralelización en la biblioteca `tokenizers`
os.environ["TOKENIZERS_PARALLELISM"] = "true"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 3ra tanda (parallismo)

In [28]:
for x in tqdm(indices_dataset_humano_cifar[100:150]):
    respuestas=predit(1,x) # 1 porque es humano
    for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_humano_respuestas.loc[x, columna] = respuesta
    del respuestas


100%|██████████| 50/50 [4:02:43<00:00, 291.27s/it]  


In [29]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte3.pkl')

In [ ]:
# Tentativa

def respuestas(x):
    #try:
        respuestas=predit(1,x) # 1 porque es humano
        for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
                #for indice_respuestas in range(0,len(respuestas)):
            dataset_humano_respuestas.loc[x, columna] = respuesta
        del respuestas
    #except:
    #    print("error")
        
#for x in tqdm(indices_dataset_humano_cifar[:50]):
    
with ThreadPoolExecutor() as executor:
    for aux in tqdm(executor.map(respuestas,indices_dataset_humano_cifar[100:150])):
        pass

# 4 Tanda

In [30]:
for x in tqdm(indices_dataset_humano_cifar[150:200]):
    respuestas=predit(1,x) # 1 porque es humano
    for columna, respuesta in zip(dataset_humano_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_humano_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:02:51<00:00, 291.43s/it]  


In [32]:
dataset_humano_respuestas.to_pickle('dataset_humano_respuestas_parte4.pkl')

In [1]:
dataset_humano_respuestas

,"USER: <image>\n""what is the background of the image?""\nASSISTANT:","USER: <image>\n""Is it rainy or sunny?""\nASSISTANT:","USER: <image>\n""Is it a closed or open place?""\nASSISTANT:","USER: <image>\n""Is the image bright?""\nASSISTANT:","USER: <image>\n""Are there elements in the image that can make visual noise?""\nASSISTANT:","USER: <image>\n""Are there a lot of different entities?""\nASSISTANT:","USER: <image>\n""Where is the image taken from?""\nASSISTANT:","USER: <image>\n""Are there humans?""\nASSISTANT:","USER: <image>\n""Are there animals?""\nASSISTANT:","USER: <image>\n""How many are there?""\nASSISTANT:",...,"USER: <image>\n""Can you give essential concepts that are relevant to the situation described?""\nASSISTANT:","USER: <image>\n""What aspects of the image have been detected that may attract attention?""\nASSISTANT:","USER: <image>\n""Why has it been obtained that the prediction is that they are ?""\nASSISTANT:","USER: <image>\n""How did you get the result that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to get the prediction that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to result in or say that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on parts of the bodies to infer that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on features or facial expressions of the to result in saying they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on the clothing to result in the being ?""\nASSISTANT:","USER: <image>\n""Do you focus on the body shape to result in the result that they are ?""\nASSISTANT:"
2106,The background of the image is a blue and purp...,Sunny,It is an open place.,"Yes, the image is bright, featuring a woman wi...","Yes, there are elements in the image that can ...","No, the image only features one woman with lon...",The image is taken from a close-up perspective...,Yes,No,"There is only one person in the image, a woman...",...,"In the image, a woman with long blonde hair is...","In the image, a woman with long blonde hair is...","The image features a woman with long hair, and...",The result that the woman in the image is a pa...,"To predict that the image features a woman, I ...","In the image, the focus is on the woman's face...","Yes, in the image, the focus is on the woman's...","Yes, the image focuses on the facial features ...","Yes, the focus on the clothing in the image, s...","Yes, the focus on the body shape and the woman..."
2131,The background of the image is blue.,It is sunny in the image.,"It is an open place, as the girl is smiling an...","Yes, the image is bright, as it is a close-up ...","Yes, there are elements in the image that can ...","No, the image only shows a single person, a yo...",The image is taken from a close-up perspective...,Yes,No,There are two people in the image.,...,"In the image, a young woman is smiling and loo...","In the image, a young woman with blonde hair i...",The image shows a young woman with blonde hair...,The result that the person in the image is a y...,To predict that the person in the image is a w...,"In the image, the focus is on the young girl's...","Yes, the image shows a close-up of a woman's f...","In the image, the girl is smiling and has a br...","Yes, the image focuses on the girl's clothing,...","Yes, the image shows a young woman with a smil..."
2179,"The background of the image is blurry, which s...",It is sunny in the image.,"It is an open place, as the man is standing in...","Yes, the image is bright, as it is described a...","Yes, there are elements in the image that can ...","Yes, there are many different entities in the ...",The image is taken from the perspective of the...,"Yes, there are humans in the image.",No,"There are several people in the image, includi...",...,"In the image, a man is holding a camera and lo...","In the image, a man is holding a camera, which...","The image shows a man holding a camera, with a...",The result that the people in the image are 

# 5 Tanda (ToDO)

# Tanda animal

In [15]:
for x in tqdm(indices_dataset_animal_cifar[:50]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:06:10<00:00, 295.41s/it]  


In [17]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte1_corregida_septiembre.pkl')

# 2da Tanda animal

In [18]:
for x in tqdm(indices_dataset_animal_cifar[50:102]): ## coloco 102 por si acaso
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 52/52 [4:20:51<00:00, 300.99s/it]  


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte2_corregida_septiembre.pkl')

In [ ]:
dataset_animales_respuestas

# 3ra Tanda animal

In [15]:
for x in tqdm(indices_dataset_animal_cifar[100:150]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:09:10<00:00, 299.01s/it]  


In [ ]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte3.pkl')

# 4ra Tanda animal (me quedo aqui para ejecutar)

In [18]:
for x in tqdm(indices_dataset_animal_cifar[150:200]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 50/50 [4:03:07<00:00, 291.76s/it]  


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte4.pkl')

# Tanda pruebas animal (10 muestras)

In [15]:
for x in tqdm(indices_dataset_animal_cifar[:10]):
    respuestas=predit(2,x) # 2 porque animal
    for columna, respuesta in zip(dataset_animales_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_animales_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 10/10 [48:06<00:00, 288.66s/it]


In [19]:
dataset_animales_respuestas.to_pickle('dataset_animal_respuestas_parte_muestras.pkl')

# Tanda pruebas objeto (10 muestras)

In [16]:
for x in tqdm(indices_dataset_objeto_cifar[:10]):
    respuestas=predit(3,x) # 3 porque objeto
    for columna, respuesta in zip(dataset_objetos_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_objetos_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 10/10 [45:38<00:00, 273.86s/it]


In [17]:
dataset_objetos_respuestas.to_pickle('dataset_objeto_respuestas_parte_muestras.pkl')

# Tanda pruebas paisaje (10 muestras) actualmento 51

In [21]:
for x in tqdm(indices_dataset_paisaje_cifar[:51]): # 51 por si acaso
    respuestas=predit(4,x) # 4 porque es paisaje
    for columna, respuesta in zip(dataset_paisajes_respuestas.columns, respuestas):#dataset_humano_respuestas.columns:
            #for indice_respuestas in range(0,len(respuestas)):
        dataset_paisajes_respuestas.loc[x, columna] = respuesta
    del respuestas

100%|██████████| 51/51 [3:38:41<00:00, 257.27s/it]  


In [22]:
dataset_paisajes_respuestas.to_pickle('dataset_paisaje_respuestas_parte_1_corregido_septiembre.pkl')

# Empezamos a trabajar con los dataset de respuestas

In [ ]:
#Dataset_respuestas_Humano
dataset_humano_respuestas_parte1=unpickle('1.0 Humano/dataset_humano_respuestas_parte1_new.pkl')
dataset_humano_respuestas_parte2=unpickle('1.0 Humano/dataset_humano_respuestas_parte2_new.pkl')
dataset_humano_respuestas_parte3=unpickle('1.0 Humano/dataset_humano_respuestas_parte3_new.pkl')
dataset_humano_respuestas_parte4=unpickle('1.0 Humano/dataset_humano_respuestas_parte4_new.pkl')
dataset_humano_respuestas_parte5=unpickle('1.0 Humano/dataset_humano_respuestas_parte5_new.pkl')



#Dataset_respuestas_Animal
dataset_animal_respuestas_parte1=unpickle('dataset_animal_respuestas_parte1_corregida_septiembre.pkl')
dataset_animal_respuestas_parte2=unpickle('dataset_animal_respuestas_parte2_corregida_septiembre.pkl')
dataset_animal_respuestas_parte3=unpickle('2.0 Animal/dataset_animal_respuestas_parte3.pkl')
dataset_animal_respuestas_parte4=unpickle('2.0 Animal/dataset_animal_respuestas_parte4.pkl')
dataset_animal_respuestas_parte5=unpickle('2.0 Animal/dataset_animal_respuestas_parte5.pkl')


#Dataset_respuestas_objeto
dataset_objeto_respuestas_parte1=unpickle('3.0 objeto/datset_objeto_respuetas_parte1.pkl')
#dataset_objeto_respuestas_parte_desconocida=unpickle('/kaggle/input/3-0dataset-categorias-preguntas-realizadas/3.0 objeto/Parece_que_con_parte5_ya_tiene_todo')
dataset_objeto_respuestas_parte3=unpickle('3.0 objeto/dataset_objeto_respuestas_parte3.pkl')
dataset_objeto_respuestas_parte4=unpickle('3.0 objeto/dataset_objeto_respuestas_parte4.pkl')
dataset_objeto_respuestas_parte5=unpickle('3.0 objeto/dataset_objeto_respuestas_parte5.pkl')

#Dataset_respuestas_paisaje
dataset_paisaje_parte1=unpickle('dataset_animal_respuestas_parte1_corregida_septiembre.pkl')
dataset_paisaje_parte2=unpickle('4.0 paisaje/dataset_paisajes_respuestas_parte2.pkl')
dataset_paisaje_parte3=unpickle('4.0 paisaje/dataset_paisajes_respuestas_parte3.pkl')
dataset_paisaje_parte4=unpickle('4.0 paisaje/dataset_paisajes_respuestas_parte4.pkl')
dataset_paisaje_parte5=unpickle('4.0 paisaje/dataset_paisajes_respuestas_parte5.pkl')

## comprobaciones humano

In [ ]:
#Correccion parte 1: 50 parte2:100 parte3:50 parte4:100 parte5: 50

In [15]:
dataset_humano_respuestas_parte1.index

Index([  58,   65,   77,   90,   92,  122,  142,  159,  188,  190,  202,  203,
        259,  260,  267,  308,  323,  382,  401,  455,  490,  493,  562,  578,
        580,  597,  598,  614,  637,  657,  666,  692,  707,  740,  745,  767,
        795,  828,  860,  879,  885,  888,  929,  968, 1003, 1021, 1067, 1078,
       1106, 1109],
      dtype='int64')

In [9]:
dataset_humano_respuestas_parte2=dataset_humano_respuestas_parte2[dataset_humano_respuestas_parte2.index>1109] # deberia ser a partir de indice 1146

In [6]:
dataset_humano_respuestas_parte2.index

Index([1146, 1195, 1198, 1228, 1266, 1267, 1291, 1303, 1311, 1328, 1366, 1451,
       1455, 1473, 1511, 1576, 1648, 1664, 1689, 1693, 1696, 1705, 1757, 1772,
       1778, 1784, 1798, 1804, 1837, 1852, 1853, 1860, 1861, 1870, 1871, 1873,
       1879, 1895, 1899, 1903, 1908, 1992, 2009, 2016, 2026, 2047, 2051, 2073,
       2088, 2103],
      dtype='int64')

In [26]:
dataset_humano_respuestas_parte3.index # sta bien

Index([2106, 2131, 2179, 2192, 2201, 2210, 2229, 2317, 2352, 2383, 2399, 2404,
       2440, 2477, 2570, 2585, 2652, 2668, 2689, 2706, 2720, 2729, 2741, 2776,
       2827, 2832, 2858, 2894, 2902, 2904, 2925, 2933, 2954, 2999, 3011, 3012,
       3072, 3112, 3125, 3131, 3137, 3147, 3254, 3272, 3289, 3345, 3349, 3351,
       3456, 3465],
      dtype='int64')

In [10]:
dataset_humano_respuestas_parte4=dataset_humano_respuestas_parte4[dataset_humano_respuestas_parte4.index>3465]

In [7]:
dataset_humano_respuestas_parte4.index

Index([3517, 3613, 3624, 3633, 3677, 3679, 3725, 3766, 3821, 3877, 3878, 3891,
       3931, 3973, 3998, 4020, 4031, 4099, 4106, 4179, 4183, 4198, 4203, 4271,
       4331, 4366, 4382, 4429, 4451, 4481, 4487, 4489, 4500, 4515, 4555, 4568,
       4621, 4685, 4747, 4750, 4773, 4859, 4861, 4929, 4934, 4979, 4994, 5019,
       5028, 5039],
      dtype='int64')

In [23]:
dataset_humano_respuestas_parte5.index

Index([5049, 5063, 5164, 5180, 5185, 5196, 5202, 5224, 5226, 5229, 5240, 5296,
       5299, 5335, 5342, 5350, 5378, 5390, 5406, 5417, 5447, 5455, 5459, 5480,
       5487, 5564, 5582, 5584, 5617, 5650, 5661, 5674, 5738, 5776, 5784, 5802,
       5853, 5895, 5913, 5921, 5965, 5997, 6021, 6058, 6097, 6149, 6159, 6216,
       6223, 6313],
      dtype='int64')

# comprobaciones animal

In [ ]:
# parte1: 50 parte2: 100 parte3: 50 parte4: 100 parte5: 50

In [5]:
dataset_animal_respuestas_parte1.index

Index([  0,   1,   4,   9,  17,  18,  26,  28,  29,  33,  40,  41,  43,  49,
        50,  63,  68,  72,  79,  81,  82,  85,  86,  87,  91,  96,  97,  98,
        99, 103, 105, 111, 121, 124, 125, 127, 129, 136, 137, 143, 146, 147,
       150, 153, 155, 158, 160, 163, 165, 169],
      dtype='int64')

In [11]:
dataset_animal_respuestas_parte2= dataset_animal_respuestas_parte2[(dataset_animal_respuestas_parte2.index>169) & (dataset_animal_respuestas_parte2.index<340)] #deberia ser apartir de 170

In [14]:
dataset_animal_respuestas_parte2.index

Index([170, 173, 177, 180, 182, 185, 189, 191, 192, 201, 205, 206, 210, 213,
       216, 224, 233, 238, 247, 250, 262, 265, 268, 271, 272, 278, 280, 282,
       284, 287, 292, 293, 296, 297, 301, 302, 305, 306, 310, 312, 313, 316,
       326, 327, 329, 333, 334, 335, 337, 338],
      dtype='int64')

In [50]:
dataset_animal_respuestas_parte3.index

Index([340, 343, 344, 346, 351, 357, 361, 364, 365, 368, 369, 372, 375, 376,
       391, 394, 396, 399, 410, 411, 422, 426, 428, 430, 432, 435, 444, 446,
       447, 448, 449, 453, 457, 458, 461, 463, 470, 474, 483, 491, 499, 502,
       508, 514, 520, 523, 525, 527, 529, 530],
      dtype='int64')

In [12]:
dataset_animal_respuestas_parte4=dataset_animal_respuestas_parte4[dataset_animal_respuestas_parte4.index>530]

In [11]:
dataset_animal_respuestas_parte4.index #deberia ser a partir de 531

Index([531, 533, 537, 539, 543, 546, 549, 552, 556, 577, 579, 589, 593, 594,
       595, 613, 615, 616, 622, 623, 627, 628, 634, 642, 643, 645, 648, 655,
       656, 661, 662, 664, 677, 678, 683, 684, 686, 691, 695, 697, 698, 708,
       711, 712, 715, 718, 724, 726, 729, 730],
      dtype='int64')

In [52]:
dataset_animal_respuestas_parte5.index

Index([732, 733, 734, 742, 750, 757, 762, 769, 770, 777, 778, 790, 793, 794,
       797, 800, 802, 808, 812, 813, 816, 817, 818, 821, 826, 829, 832, 833,
       836, 842, 846, 848, 852, 859, 866, 869, 870, 876, 884, 886, 890, 892,
       893, 898, 903, 904, 905, 906, 910, 914],
      dtype='int64')

# comprobaciones objeto

In [ ]:
#parte 1: 50 parte 2: NA parte 3: 150 parte 4: 200 parte 5: 250

In [76]:
len(pd.Series(dataset_objeto_respuestas_parte5.index.tolist()).unique())

# En esta ocasion ya la parte 5 contiene todo el dataset

250

# comprobaciones paisaje

In [ ]:
#parte 1 250, parte2: 100 parte3: 150 parte4: 200 parte5: 250 yo creo que con la parte 5 buscar porque:: porque tiene 48 preguntas que es lo correcto.

In [40]:
dataset_paisaje_parte5.head(5)

,"USER: <image>\n""what is the background of the image?""\nASSISTANT:","USER: <image>\n""Is it rainy or sunny?""\nASSISTANT:","USER: <image>\n""Is it a closed or open place?""\nASSISTANT:","USER: <image>\n""Is the image bright?""\nASSISTANT:","USER: <image>\n""Are there elements in the image that can make visual noise?""\nASSISTANT:","USER: <image>\n""Are there a lot of different entities?""\nASSISTANT:","USER: <image>\n""Where is the image taken from?""\nASSISTANT:","USER: <image>\n""Are there humans?""\nASSISTANT:","USER: <image>\n""Are there animals?""\nASSISTANT:","USER: <image>\n""How many are there?""\nASSISTANT:",...,"USER: <image>\n""Could you in simpler words describe their function that you have predicted before?""\nASSISTANT:","USER: <image>\n""What could stand out from the situation described above and be useful in similar situations?""\nASSISTANT:","USER: <image>\n""Can you give essential concepts that are relevant to the situation described?""\nASSISTANT:","USER: <image>\n""What aspects of the image have been detected that may attract attention?""\nASSISTANT:","USER: <image>\n""Why has it been obtained that the prediction is that they are ?""\nASSISTANT:","USER: <image>\n""How did you get the result that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to get the prediction that they are ?""\nASSISTANT:","USER: <image>\n""What do you focus on to result in or say that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on parts of the bodies to infer that they are ?""\nASSISTANT:","USER: <image>\n""Do you focus on the body shape to result in the result that they are ?""\nASSISTANT:"
12,The background of the image features a green f...,"It is sunny, as the sunflowers are blooming in...","It is an open place, as there are several sunf...","Yes, the image is bright, as it features a sun...","Yes, there are elements in the image that can ...","No, there are not a lot of different entities ...","The image is taken from a low angle, looking u...",No,No,There are three yellow sunflowers in the image.,...,The sunflowers in the image are designed to at...,"In the image, a yellow sunflower is growing in...","In the image, there are three yellow sunflower...","In the image, the yellow sunflowers have been ...",The image shows a group of yellow sunflowers i...,The result that the sunflowers are yellow is a...,To predict that the yellow flowers are sunflow...,To result in or say that the sunflowers are ye...,"Yes, in the image, the focus is on the yellow ...","Yes, the focus on the body shape of the sunflo..."
14,The background of the image features a beautif...,It is sunny.,"It is an open place, as it is a beach with the...","Yes, the image is bright, as it features a bea...","Yes, there are elements in the image that can ...","No, there are not a lot of different entities ...","The image is taken from a beach, overlooking t...",No,No,There are two boats in the image.,...,The sun is a star that provides light and warm...,"In the image, a boat is seen in the distance, ...","In the image, a boat is sailing on a large bod...","In the image, the sun is setting over the ocea...",The image shows a beautiful sunset over the oc...,I achieved the result that the people are stan...,"To get the prediction that the sun is setting,...","To result in or say that the sun is setting, o...","Yes, I focus on parts of the bodies to infer t...","Yes, the image focuses on the body shape of th..."
19,The background of the image features a clear b...,"It is sunny in the image, as the sky is clear ...","It is an open place, as it is described as a l...","Yes, the image is bright, as it features a cle...","Yes, there are elements in the image that can ...","No, the image features a single entity, which ...","The image is taken from a beach, overlooking t...",No,No,There are two clouds in the image.,...,The clouds in the image are floating in the sk...,"In the image, a cloud is visible in the sky, w...","In the image, there is a c

## Tratamiento de cabeceras (eliminar los caracteres especiales de la columna)

### Para dataset respuestas humanos 

In [ ]:
def convertir_cabecera(x):
    string_lista=re.findall('"([^"]*)"',x)
    return string_lista[0]

nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_humano_respuestas_parte5.columns.tolist()]


dataset_humano_respuestas_parte1.columns=nuevas_cabeceras_1
dataset_humano_respuestas_parte2.columns=nuevas_cabeceras_2
dataset_humano_respuestas_parte3.columns=nuevas_cabeceras_3
dataset_humano_respuestas_parte4.columns=nuevas_cabeceras_4
dataset_humano_respuestas_parte5.columns=nuevas_cabeceras_5

### Para dataset respuestas animales 

In [16]:
#La parte 1 y 2 ya tienen las cabeceras actualizadas

nuevas_cabeceras_1=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte1.columns.tolist()]
nuevas_cabeceras_2=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte2.columns.tolist()]
nuevas_cabeceras_3=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte3.columns.tolist()]
nuevas_cabeceras_4=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte4.columns.tolist()]
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_animal_respuestas_parte5.columns.tolist()]


dataset_animal_respuestas_parte1.columns=nuevas_cabeceras_1
dataset_animal_respuestas_parte2.columns=nuevas_cabeceras_2
dataset_animal_respuestas_parte3.columns=nuevas_cabeceras_3
dataset_animal_respuestas_parte4.columns=nuevas_cabeceras_4
dataset_animal_respuestas_parte5.columns=nuevas_cabeceras_5


In [ ]:
print(dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte1.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte2.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte3.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte4.columns==dataset_animal_respuestas_parte5.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte1.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte2.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte3.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte4.columns,
dataset_animal_respuestas_parte5.columns==dataset_animal_respuestas_parte5.columns)

# resultado todo true

### Para dataset respuestas objetos 

In [17]:
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_objeto_respuestas_parte5.columns.tolist()]

dataset_objeto_respuestas_parte5.columns=nuevas_cabeceras_5

### Para dataset respuestas paisajes 

In [18]:
nuevas_cabeceras_5=[convertir_cabecera(x) for x in dataset_paisaje_parte5.columns.tolist()]


dataset_paisaje_parte5.columns=nuevas_cabeceras_5

## Caso paisaje el dataset solo es la parte 5, Caso objeto el dataset solo es la parte 5.

# Union dataset_respuestas y cabeceras (etiqueta_singularidad....)

In [ ]:
# Union de los dataset respuestas que estan en pequeñas partes en uno solo

dataset_humano_respuestas= pd.concat([dataset_humano_respuestas_parte1,
                                        dataset_humano_respuestas_parte2,
                                        dataset_humano_respuestas_parte3,
                                        dataset_humano_respuestas_parte4,
                                        dataset_humano_respuestas_parte5], ignore_index=False)

dataset_animal_respuestas=pd.concat([dataset_animal_respuestas_parte1,
                                    dataset_animal_respuestas_parte2,
                                    dataset_animal_respuestas_parte3,
                                    dataset_animal_respuestas_parte4,
                                    dataset_animal_respuestas_parte5], ignore_index=False)


# para este caso no es necesario concatenar
dataset_objeto_respuestas=dataset_objeto_respuestas_parte5


# para este caso no es necesario concatenar
dataset_paisaje_respuestas=dataset_paisaje_parte5

In [20]:
# para evitar problemas de clave en el join de tablas

dataset_humano_cifar.loc[:,'indice_2']=dataset_humano_cifar.index
dataset_humano_respuestas.loc[:,'indice_2']=dataset_humano_respuestas.index

dataset_animal_cifar.loc[:,'indice_2']=dataset_animal_cifar.index
dataset_animal_respuestas.loc[:,'indice_2']=dataset_animal_respuestas.index

dataset_objeto_cifar.loc[:,'indice_2']=dataset_objeto_cifar.index
dataset_objeto_respuestas.loc[:,'indice_2']=dataset_objeto_respuestas.index

dataset_paisaje_cifar.loc[:,'indice_2']=dataset_paisaje_cifar.index
dataset_paisaje_respuestas.loc[:,'indice_2']=dataset_paisaje_respuestas.index


In [15]:
# funcion sql
def sql_query(query):
    q=sql.sqldf
    consulta=q(query,globals())
    return consulta

In [39]:
query_humano='''
SELECT dataset_humano_cifar."b'data'", dataset_humano_cifar.etiqueta, dataset_humano_cifar.singularidad, dataset_humano_cifar.categoria,  dataset_humano_respuestas.* FROM dataset_humano_cifar, dataset_humano_respuestas
WHERE dataset_humano_cifar.indice_2=dataset_humano_respuestas.indice_2
'''

query_animal='''
SELECT dataset_animal_cifar."b'data'", dataset_animal_cifar.etiqueta, dataset_animal_cifar.singularidad, dataset_animal_cifar.categoria,  dataset_animal_respuestas.* FROM dataset_animal_cifar, dataset_animal_respuestas
WHERE dataset_animal_cifar.indice_2=dataset_animal_respuestas.indice_2
'''

query_objeto='''
SELECT dataset_objeto_cifar."b'data'", dataset_objeto_cifar.etiqueta, dataset_objeto_cifar.singularidad, dataset_objeto_cifar.categoria,  dataset_objeto_respuestas.* FROM dataset_objeto_cifar, dataset_objeto_respuestas
WHERE dataset_objeto_cifar.indice_2=dataset_objeto_respuestas.indice_2
'''

query_paisaje='''
SELECT dataset_paisaje_cifar."b'data'", dataset_paisaje_cifar.etiqueta, dataset_paisaje_cifar.singularidad, dataset_paisaje_cifar.categoria,  dataset_paisaje_respuestas.* FROM dataset_paisaje_cifar, dataset_paisaje_respuestas
WHERE dataset_paisaje_cifar.indice_2=dataset_paisaje_respuestas.indice_2
'''

dataset_humano_respuestas_completo=sql_query(query_humano)
dataset_animal_respuestas_completo=sql_query(query_animal)
dataset_objeto_respuestas_completo=sql_query(query_objeto)
dataset_paisaje_respuestas_completo=sql_query(query_paisaje)

In [40]:
indices_singular_humano=dataset_humano_respuestas_completo[dataset_humano_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_animal=dataset_animal_respuestas_completo[dataset_animal_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_objeto=dataset_objeto_respuestas_completo[dataset_objeto_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()
indices_singular_paisaje=dataset_paisaje_respuestas_completo[dataset_paisaje_respuestas_completo['singularidad']=='singular']['indice_2'].tolist()

In [41]:
# resetear index
dataset_humano_respuestas_completo.set_index(dataset_humano_respuestas_completo['indice_2'],inplace=True)
dataset_humano_respuestas_completo = dataset_humano_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_animal_respuestas_completo.set_index(dataset_animal_respuestas_completo['indice_2'],inplace=True)
dataset_animal_respuestas_completo = dataset_animal_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_objeto_respuestas_completo.set_index(dataset_objeto_respuestas_completo['indice_2'],inplace=True)
dataset_objeto_respuestas_completo = dataset_objeto_respuestas_completo.rename(columns={'indice_2': 'indice'})

# resetear index
dataset_paisaje_respuestas_completo.set_index(dataset_paisaje_respuestas_completo['indice_2'],inplace=True)
dataset_paisaje_respuestas_completo = dataset_paisaje_respuestas_completo.rename(columns={'indice_2': 'indice'})

In [42]:
# Colocamos 0000 para identificar que es nulo, esto porque np.nan uego da problemas para manipulacion explotacion.

#columnas plularidad humano
dataset_humano_respuestas_completo.loc[indices_singular_humano,[' have similar colors?','Are  far away from each other?','Do  interact with each other?']]=0000

#columnas plularidad animal
dataset_animal_respuestas_completo.loc[indices_singular_animal,[' have similar colors?','Are  far away from each other?','Do  interact with each other?']]=0000


#columnas plularidad objeto
dataset_objeto_respuestas_completo.loc[indices_singular_objeto,[' have similar colors?', "Are  far away from each other?"]]=0000


#columnas plularidad paisaje
dataset_paisaje_respuestas_completo.loc[indices_singular_paisaje,' have similar colors?']=0000

# Filtro numero 1

## Para obtener las categorias y singularidades presentes a trabajar

In [52]:
#agrupacion por etiqueta y singularidad
# AQUI SOLO HAY HUMANOS CREO QUE SERIA PARA EL RESTO DE DATASET TAMBIEN
# SERIA AGREGAR LOS UNICOS DEL RESTO DE DATASETS
query_unicos_humanos= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_humano_respuestas_completo
'''

query_unicos_animal= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_animal_respuestas_completo
'''

query_unicos_objeto= '''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_objeto_respuestas_completo
'''

query_unicos_paisaje='''
SELECT DISTINCT(ETIQUETA), SINGULARIDAD FROM dataset_paisaje_respuestas_completo
'''

unicos_humanos= sql_query(query_unicos_humanos)
unicos_animal= sql_query(query_unicos_animal)
unicos_objetos= sql_query(query_unicos_objeto)
unicos_paisajes= sql_query(query_unicos_paisaje)

In [53]:
unicos_humanos

,etiqueta,singularidad
0,woman,singular
1,girl,singular
2,man,singular
3,baby,singular


In [54]:
unicos_animal

,etiqueta,singularidad
0,cattle,plural
1,dinosaur,singular
2,aquarium_fish,singular
3,elephant,singular
4,wolf,singular
5,squirrel,singular
6,possum,singular
7,leopard,singular
8,rabbit,singular
9,snake,singular


In [78]:
unicos_objetos

,etiqueta,singularidad
0,apple,singular
1,telephone,singular
2,cup,singular
3,bicycle,singular
4,television,singular
5,table,singular
6,clock,singular
7,streetcar,singular
8,tractor,singular
9,chair,singular


In [83]:
unicos_paisajes

,etiqueta,singularidad
0,sunflower,singular
1,sea,singular
2,pine_tree,singular
3,mountain,singular
4,forest,singular
5,orchid,singular


# para tener los subsatadets por etiqueta y singularidad

## Caso Humanos

In [46]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

# funciones de metricas

In [29]:
%pip install textdistance

Note: you may need to restart the kernel to use updated packages.


In [50]:
import textdistance

# Levestehin
def similitud_metrica_levestehin(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.levenshtein(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# overlap coefficient
def similitud_metrica_overlap(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.overlap(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# cosine similarity
def similitud_metrica_cosine(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.cosine(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666
    

    
# Jaccard index
def similitud_metrica_jaccard(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.jaccard(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666


    
# bag distance 0 significa que es igual
def similitud_metrica_bag(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion= str(texto_iteracion)
    if texto_query!='0':
        similitud=textdistance.bag(texto_query,texto_iteracion)
        return similitud
    else:
        return 6666

In [3]:
web_model = WebBertSimilarity(device='cuda',batch_size=5) #defaults to GPU prediction

In [8]:
# funcion metrica de similitud_1 semantica
def similitud_metrica_websimilarity(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion=str(texto_iteracion)
    if texto_query!='0':
        similitud=web_model.predict([(texto_query,texto_iteracion)])
        return float(similitud[0]) # va del 0 al 5
    else:
        return 6666

# estudiar el multitrheads porque es muy lento

In [49]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

# Ahora lista_comparacion contiene todos los resultados


62it [2:47:58, 162.56s/it]
67it [3:14:31, 174.20s/it]
61it [2:43:59, 161.30s/it]
60it [2:34:32, 154.53s/it]


In [50]:
df_resultado=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


In [51]:
df_resultado.to_csv('salida_prueba.csv', index=False)

## Caso animales

In [89]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [90]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [07:17, 33.65s/it]
10it [04:21, 26.17s/it]
7it [01:53, 16.16s/it]
10it [04:52, 29.20s/it]
12it [05:54, 29.50s/it]
8it [02:33, 19.20s/it]
14it [08:34, 36.72s/it]
10it [04:05, 24.58s/it]
15it [09:16, 37.09s/it]
8it [02:36, 19.57s/it]
11it [04:50, 26.38s/it]
9it [03:28, 23.20s/it]
9it [03:20, 22.29s/it]
9it [03:26, 22.90s/it]
7it [02:00, 17.28s/it]
8it [02:31, 18.90s/it]
17it [12:19, 43.53s/it]
8it [02:35, 19.40s/it]
4it [00:32,  8.16s/it]
6it [01:16, 12.69s/it]
10it [04:01, 24.13s/it]
6it [01:26, 14.38s/it]
7it [01:58, 16.97s/it]
11it [05:07, 27.96s/it]
10it [04:03, 24.38s/it]
6it [01:21, 13.64s/it]
5it [00:53, 10.77s/it]


In [91]:
df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [92]:
df_resultado_animales.to_csv('salida_prueba_animal_2.csv', index=False)

## Caso objetos

In [79]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [80]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [05:41, 28.42s/it]
9it [03:20, 22.24s/it]
11it [04:37, 25.22s/it]
14it [08:31, 36.56s/it]
15it [09:33, 38.20s/it]
18it [14:24, 48.00s/it]
9it [03:10, 21.16s/it]
12it [06:33, 32.77s/it]
16it [12:32, 47.02s/it]
12it [05:14, 26.21s/it]
14it [08:22, 35.89s/it]
10it [04:51, 29.18s/it]
10it [03:36, 21.67s/it]
16it [10:35, 39.75s/it]
12it [06:23, 31.94s/it]
12it [05:51, 29.28s/it]
10it [04:02, 24.24s/it]
5it [00:51, 10.22s/it]
9it [03:05, 20.62s/it]
16it [09:54, 37.18s/it]
8it [02:37, 19.71s/it]


In [81]:
df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [84]:
df_resultado_objeto.to_csv('salida_prueba_objeto_2.csv', index=False)

## Caso paisajes

In [85]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [86]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [51:57, 89.07s/it]
52it [1:57:57, 136.10s/it]
39it [1:04:38, 99.44s/it] 
39it [1:14:12, 114.16s/it]
48it [1:42:50, 128.55s/it]
37it [55:05, 89.35s/it]


In [87]:
df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])

In [88]:
df_resultado_paisaje.to_csv('salida_prueba_paisaje.csv', index=False)

# Ahora empezamos el procedimiento con la similitud overlap

# Caso humano overlap

In [93]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_overlap.csv', index=False)


62it [00:08,  7.35it/s]
67it [00:09,  7.04it/s]
61it [00:08,  7.50it/s]
60it [00:07,  7.52it/s]


# Caso animales overlap

In [94]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_animales.to_csv('salida_animal_overlap.csv', index=False)

13it [00:00, 33.10it/s]
10it [00:00, 44.71it/s]
7it [00:00, 63.60it/s]
10it [00:00, 43.67it/s]
12it [00:00, 38.28it/s]
8it [00:00, 54.58it/s]
14it [00:00, 25.95it/s]
10it [00:00, 45.44it/s]
15it [00:00, 31.26it/s]
8it [00:00, 53.58it/s]
11it [00:00, 41.81it/s]
9it [00:00, 49.33it/s]
9it [00:00, 50.29it/s]
9it [00:00, 48.64it/s]
7it [00:00, 62.05it/s]
8it [00:00, 45.51it/s]
17it [00:00, 25.80it/s]
8it [00:00, 53.09it/s]
4it [00:00, 37.40it/s]
6it [00:00, 67.65it/s]
10it [00:00, 41.82it/s]
6it [00:00, 71.67it/s]
7it [00:00, 63.40it/s]
11it [00:00, 37.52it/s]
10it [00:00, 43.17it/s]
6it [00:00, 71.18it/s]
5it [00:00, 83.62it/s]


# Caso objeto overlap 

In [95]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_overlap.csv', index=False) # modificar

12it [00:00, 39.79it/s]
9it [00:00, 52.69it/s]
11it [00:00, 42.78it/s]
14it [00:00, 35.13it/s]
15it [00:00, 29.40it/s]
18it [00:00, 25.41it/s]
9it [00:00, 48.12it/s]
12it [00:00, 35.31it/s]
16it [00:00, 29.03it/s]
12it [00:00, 42.73it/s]
14it [00:00, 33.29it/s]
10it [00:00, 45.99it/s]
10it [00:00, 38.67it/s]
16it [00:00, 31.05it/s]
12it [00:00, 40.10it/s]
12it [00:00, 35.95it/s]
10it [00:00, 47.51it/s]
5it [00:00, 89.39it/s]
9it [00:00, 51.57it/s]
16it [00:00, 31.40it/s]
8it [00:00, 58.54it/s]


# Caso paisaje overlap

In [96]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_overlap.csv', index=False) # modificar

35it [00:02, 14.84it/s]
52it [00:05, 10.33it/s]
39it [00:02, 13.97it/s]
39it [00:02, 13.24it/s]
48it [00:04, 11.34it/s]
37it [00:02, 14.04it/s]


# Ahora empezamos el procedimiento con la similitud cosine

# Caso humano cosine

In [271]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_cosine.csv', index=False)


0it [00:00, ?it/s]


NameError: name 'similitud_metrica_cosine' is not defined

# Caso animal cosine

In [98]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_cosine.csv', index=False) # modificar

13it [00:00, 29.25it/s]
10it [00:00, 43.98it/s]
7it [00:00, 61.22it/s]
10it [00:00, 40.15it/s]
12it [00:00, 36.83it/s]
8it [00:00, 52.92it/s]
14it [00:00, 31.33it/s]
10it [00:00, 44.40it/s]
15it [00:00, 30.16it/s]
8it [00:00, 54.57it/s]
11it [00:00, 40.40it/s]
9it [00:00, 37.90it/s]
9it [00:00, 41.01it/s]
9it [00:00, 46.27it/s]
7it [00:00, 57.92it/s]
8it [00:00, 53.54it/s]
17it [00:00, 26.74it/s]
8it [00:00, 54.37it/s]
4it [00:00, 96.45it/s]
6it [00:00, 70.72it/s]
10it [00:00, 43.78it/s]
6it [00:00, 57.75it/s]
7it [00:00, 61.20it/s]
11it [00:00, 33.70it/s]
10it [00:00, 40.23it/s]
6it [00:00, 69.32it/s]
5it [00:00, 80.46it/s]


# Caso objeto cosine

In [99]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_cosine.csv', index=False) # modificar

12it [00:00, 39.83it/s]
9it [00:00, 52.28it/s]
11it [00:00, 45.27it/s]
14it [00:00, 35.37it/s]
15it [00:00, 32.96it/s]
18it [00:00, 27.31it/s]
9it [00:00, 52.34it/s]
12it [00:00, 33.50it/s]
16it [00:00, 30.07it/s]
12it [00:00, 41.37it/s]
14it [00:00, 34.39it/s]
10it [00:00, 46.34it/s]
10it [00:00, 47.80it/s]
16it [00:00, 30.81it/s]
12it [00:00, 38.08it/s]
12it [00:00, 36.51it/s]
10it [00:00, 40.48it/s]
5it [00:00, 84.54it/s]
9it [00:00, 50.30it/s]
16it [00:00, 30.41it/s]
8it [00:00, 57.92it/s]


# Caso paisaje cosine

In [100]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_cosine.csv', index=False) # modificar

35it [00:02, 15.03it/s]
52it [00:04, 10.45it/s]
39it [00:02, 13.67it/s]
39it [00:02, 13.50it/s]
48it [00:04, 11.19it/s]
37it [00:02, 14.43it/s]


# Ahora empezamos el procedimiento con la similitud jaccard

# Caso humano jaccard

In [101]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_jaccard.csv', index=False)


62it [00:08,  7.13it/s]
67it [00:09,  6.74it/s]
61it [00:08,  7.24it/s]
60it [00:07,  7.51it/s]


# Caso animal jaccard

In [102]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_jaccard.csv', index=False) # modificar

13it [00:00, 30.95it/s]
10it [00:00, 33.83it/s]
7it [00:00, 58.31it/s]
10it [00:00, 41.27it/s]
12it [00:00, 35.62it/s]
8it [00:00, 52.25it/s]
14it [00:00, 29.42it/s]
10it [00:00, 40.15it/s]
15it [00:00, 27.49it/s]
8it [00:00, 47.11it/s]
11it [00:00, 38.24it/s]
9it [00:00, 46.49it/s]
9it [00:00, 47.01it/s]
9it [00:00, 45.85it/s]
7it [00:00, 40.05it/s]
8it [00:00, 50.77it/s]
17it [00:00, 25.49it/s]
8it [00:00, 46.71it/s]
4it [00:00, 94.62it/s]
6it [00:00, 70.86it/s]
10it [00:00, 41.49it/s]
6it [00:00, 67.41it/s]
7it [00:00, 56.88it/s]
11it [00:00, 38.21it/s]
10it [00:00, 42.20it/s]
6it [00:00, 68.07it/s]
5it [00:00, 42.43it/s]


# Caso objeto jaccard

In [103]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_jaccard.csv', index=False) # modificar

12it [00:00, 37.70it/s]
9it [00:00, 47.28it/s]
11it [00:00, 41.29it/s]
14it [00:00, 32.80it/s]
15it [00:00, 30.94it/s]
18it [00:00, 25.29it/s]
9it [00:00, 49.95it/s]
12it [00:00, 36.99it/s]
16it [00:00, 27.98it/s]
12it [00:00, 35.32it/s]
14it [00:00, 32.57it/s]
10it [00:00, 35.38it/s]
10it [00:00, 45.86it/s]
16it [00:00, 29.44it/s]
12it [00:00, 34.12it/s]
12it [00:00, 30.86it/s]
10it [00:00, 45.55it/s]
5it [00:00, 88.47it/s]
9it [00:00, 50.34it/s]
16it [00:00, 28.69it/s]
8it [00:00, 55.14it/s]


# Caso paisaje jaccard

In [104]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_jaccard.csv', index=False) # modificar

35it [00:02, 14.32it/s]
52it [00:05,  9.63it/s]
39it [00:03, 12.70it/s]
39it [00:03, 12.08it/s]
48it [00:04, 10.19it/s]
37it [00:02, 13.68it/s]


# Ahora empezamos el procedimiento con la similitud bag

# Caso humano bag

In [105]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_humano=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion'])


df_resultado_humano.to_csv('salida_humano_bag.csv', index=False)

62it [00:09,  6.40it/s]
67it [00:11,  6.08it/s]
61it [00:09,  6.54it/s]
60it [00:08,  6.83it/s]


# Caso animal bag

In [106]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])
    


# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_animales=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animales.to_csv('salida_animal_bag.csv', index=False) # modificar

13it [00:00, 28.39it/s]
10it [00:00, 31.08it/s]
7it [00:00, 51.11it/s]
10it [00:00, 37.95it/s]
12it [00:00, 32.64it/s]
8it [00:00, 47.57it/s]
14it [00:00, 27.86it/s]
10it [00:00, 37.51it/s]
15it [00:00, 23.50it/s]
8it [00:00, 47.77it/s]
11it [00:00, 35.98it/s]
9it [00:00, 39.15it/s]
9it [00:00, 43.66it/s]
9it [00:00, 43.40it/s]
7it [00:00, 54.48it/s]
8it [00:00, 48.72it/s]
17it [00:00, 23.60it/s]
8it [00:00, 47.96it/s]
4it [00:00, 88.10it/s]
6it [00:00, 60.34it/s]
10it [00:00, 36.67it/s]
6it [00:00, 38.83it/s]
7it [00:00, 52.52it/s]
11it [00:00, 33.99it/s]
10it [00:00, 36.02it/s]
6it [00:00, 61.77it/s]
5it [00:00, 73.63it/s]


# Caso objeto bag

In [114]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_objeto=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto.to_csv('salida_objeto_bag.csv', index=False) # modificar

12it [00:00, 34.79it/s]
9it [00:00, 40.92it/s]
11it [00:00, 37.59it/s]
14it [00:00, 29.04it/s]
15it [00:00, 26.49it/s]
18it [00:00, 23.42it/s]
9it [00:00, 46.33it/s]
12it [00:00, 34.68it/s]
16it [00:00, 26.02it/s]
12it [00:00, 36.40it/s]
14it [00:00, 30.43it/s]
10it [00:00, 40.28it/s]
10it [00:00, 42.60it/s]
16it [00:00, 23.19it/s]
12it [00:00, 34.98it/s]
12it [00:00, 34.75it/s]
10it [00:00, 41.66it/s]
5it [00:00, 81.12it/s]
9it [00:00, 46.63it/s]
16it [00:00, 27.18it/s]
8it [00:00, 51.69it/s]


# Dataset paisaje bag

In [115]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])
    

# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)


# aqui guardamos la metrica

df_resultado_paisaje=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje.to_csv('salida_paisaje_bag.csv', index=False) # modificar


35it [00:02, 13.38it/s]
52it [00:05,  9.04it/s]
39it [00:03, 11.71it/s]
39it [00:03, 11.19it/s]
48it [00:04,  9.63it/s]
37it [00:03, 12.14it/s]


In [ ]:
### voy a guardar los dataset completos
dataset_humano_respuestas_completo.to_csv("dataset_humano_respuestas_completo.csv",index=True)
dataset_animal_respuestas_completo.to_csv("dataset_animal_respuestas_completo.csv",index=True)
dataset_objeto_respuestas_completo.to_csv("dataset_objeto_respuestas_completo.csv",index=True)
dataset_paisaje_respuestas_completo.to_csv("dataset_paisaje_respuestas_completo.csv",index=True)

## voy a guardar los unicos
unicos_humanos.to_csv("unicos_humanos.csv",index=True)
unicos_animal.to_csv("unicos_animal.csv",index=True)
unicos_objetos.to_csv("unicos_objetos.csv",index=True)
unicos_paisajes.to_csv("unicos_paisajes.csv",index=True)


In [5]:
unicos_humanos=pd.read_csv("unicos_humanos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_animal=pd.read_csv("unicos_animal.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_objetos=pd.read_csv("unicos_objetos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_paisajes=pd.read_csv("unicos_paisajes.csv",usecols=lambda columna: columna != 'Unnamed: 0')

dataset_humano_respuestas_completo=pd.read_csv("dataset_humano_respuestas_completo.csv",index_col='indice_2')
dataset_animal_respuestas_completo=pd.read_csv("dataset_animal_respuestas_completo.csv",index_col='indice_2')
dataset_objeto_respuestas_completo=pd.read_csv("dataset_objeto_respuestas_completo.csv",index_col='indice_2')
dataset_paisaje_respuestas_completo=pd.read_csv("dataset_paisaje_respuestas_completo.csv",index_col='indice_2')

# TODO metricas de semantica (modelos), metrica de imagenes

# Filtro 2 (Counterfractuals)

# Humano levestehin

In [5]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [6]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)


0it [00:00, ?it/s]

0 58


1it [07:58, 478.31s/it]

1 188


2it [16:40, 504.31s/it]

...

58it [8:13:47, 502.04s/it]

58 6223


59it [8:22:49, 513.84s/it]

59 6313


60it [8:31:19, 511.32s/it]


In [7]:
df_resultado_humano_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_levestein.to_csv('df_resultado_humano_levestein_filtro_2.csv', index=False) # modificar

In [8]:
lista_comparacion_dosmil_minutos=lista_comparacion.copy()

In [9]:
lista_comparacion_dosmil_minutos

[(58, 65, 'Are There  male?', 0),
 (58, 92, 'Are There  male?', 0),
 (58, 142, 'Are There  male?', 38),
 (58, 455, 'Are There  male?', 0),
 (58, 597, 'Are There  male?', 0),
 (58, 745, 'Are There  male?', 0),
 (58, 879, 'Are There  male?', 0),
 (58, 1003, 'Are There  male?', 46),
 (58, 1067, 'Are There  male?', 77),
 (58, 1109, 'Are There  male?', 0),
 (58, 1266, 'Are There  male?', 0),
 (58, 1303, 'Are There  male?', 0),
 (58, 1455, 'Are There  male?', 0),
 (58, 1648, 'Are There  male?', 0),
 (58, 1689, 'Are There  male?', 0),
 (58, 1693, 'Are There  male?', 0),
 (58, 1757, 'Are There  male?', 84),
 (58, 1798, 'Are There  male?', 0),
 (58, 1852, 'Are There  male?', 88),
 (58, 1870, 'Are There  male?', 0),
 (58, 1873, 'Are There  male?', 39),
 (58, 2073, 'Are There  male?', 0),
 (58, 2088, 'Are There  male?', 0),
 (58, 2201, 'Are There  male?', 0),
 (58, 2229, 'Are There  male?', 43),
 (58, 2404, 'Are There  male?', 0),
 (58, 2570, 'Are There  male?', 0),
 (58, 2585, 'Are There  male?'

# Animal levestehin

In [10]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [11]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 0


1it [01:16, 76.72s/it]

1 137


2it [02:34, 77.33s/it]

...

3it [04:16, 84.71s/it]

3 613


4it [05:35, 82.62s/it]

4 869


5it [06:59, 83.92s/it]


In [12]:
df_resultado_animal_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_levestein.to_csv('df_resultado_animal_levestein_filtro_2.csv', index=False) # modificar

# Objeto levenstehin

In [13]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [14]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 2


1it [01:25, 85.19s/it]

1 176


2it [02:50, 85.50s/it]

...

6it [08:24, 84.09s/it]

6 1179


7it [09:50, 84.72s/it]

7 1211


8it [11:15, 84.46s/it]


In [15]:
df_resultado_objeto_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_levestein.to_csv('df_resultado_objeto_levestein_filtro_2.csv', index=False) # modificar

# Paisaje levestehin

In [16]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [17]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_levestehin(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [06:35, 395.60s/it]

1 164


2it [12:26, 369.52s/it]

...

35it [3:08:07, 326.58s/it]

35 3390


36it [3:13:51, 331.88s/it]

36 3510


37it [3:19:04, 322.82s/it]


In [18]:
df_resultado_paisaje_levestein=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_levestein.to_csv('df_resultado_paisaje_levestein_filtro_2.csv', index=False) # modificar

# Humano overlap

In [47]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [20]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.54it/s]

1 188


2it [00:00,  2.64it/s]

...

58it [00:22,  2.65it/s]

58 6223


59it [00:22,  2.63it/s]

59 6313


60it [00:22,  2.62it/s]


In [21]:
df_resultado_humano_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_overlap.to_csv('df_resultado_humano_overlap_filtro_2.csv', index=False) # modificar

# Animal Overlap

In [22]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [23]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 15.30it/s]

0 0
1 137
2 180
3 224


6it [00:00, 15.78it/s]

4 278
5 351
6 430
7 529


10it [00:00, 15.99it/s]

...

2it [00:00, 14.75it/s]

2 449
3 613


4it [00:00, 14.66it/s]

4 869


5it [00:00, 14.78it/s]


In [24]:
df_resultado_animal_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_overlap.to_csv('df_resultado_animal_overlap_filtro_2.csv', index=False) # modificar

# QUEDO AQUI

# Objeto Overlap

In [51]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [7]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

1it [00:00,  7.73it/s]

0 2
1 176


3it [00:00,  8.31it/s]

2 304
3 317


5it [00:00,  8.39it/s]

...

4it [00:00, 14.55it/s]

3 465
4 806
5 963


6it [00:00, 14.36it/s]

6 1179
7 1211


8it [00:00, 14.26it/s]


In [8]:
df_resultado_objeto_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_overlap.to_csv('df_resultado_objeto_overlap_filtro_2.csv', index=False) # modificar

# Paisaje Overlap

In [53]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [54]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_overlap(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  2.54it/s]

1 164


2it [00:00,  2.57it/s]

...

35it [00:09,  3.73it/s]

35 3390


36it [00:09,  3.75it/s]

36 3510


37it [00:09,  3.88it/s]


In [55]:
df_resultado_paisaje_overlap=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_overlap.to_csv('df_resultado_paisaje_overlap_filtro_2.csv', index=False) # modificar

# Humano Cosine

In [11]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [12]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.52it/s]

1 188


2it [00:00,  2.57it/s]

...

58it [00:23,  2.54it/s]

58 6223


59it [00:23,  2.50it/s]

59 6313


60it [00:24,  2.50it/s]


In [13]:
df_resultado_humano_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_cosine.to_csv('df_resultado_humano_cosine_filtro_2.csv', index=False) # modificar

# Animal Cosine

In [14]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [15]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.73it/s]

0 0
1 137
2 180
3 224


6it [00:00, 14.82it/s]

4 278
5 351
6 430
7 529


10it [00:00, 15.19it/s]

...

0 313
1 372


2it [00:00, 15.68it/s]

2 449
3 613


5it [00:00, 15.30it/s]

4 869


In [16]:
df_resultado_animal_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_animal_cosine.to_csv('df_resultado_animal_cosine_filtro_2.csv', index=False) # modificar

# Objeto cosine

In [17]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [18]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.71it/s]

0 2
1 176
2 304


4it [00:00, 14.06it/s]

3 317
4 353
5 441


8it [00:00, 13.90it/s]

...

4it [00:00, 15.71it/s]

4 806


6it [00:00, 15.41it/s]

5 963
6 1179
7 1211


8it [00:00, 15.41it/s]


In [19]:
df_resultado_objeto_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_objeto_cosine.to_csv('df_resultado_objeto_cosine_filtro_2.csv', index=False) # modificar

# Paisaje cosine

In [20]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [21]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_cosine(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  4.06it/s]

1 164


2it [00:00,  4.08it/s]

...

35it [00:07,  4.40it/s]

35 3390


36it [00:08,  4.33it/s]

36 3510


37it [00:08,  4.40it/s]


In [22]:
df_resultado_paisaje_cosine=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_paisaje_cosine.to_csv('df_resultado_paisaje_cosine_filtro_2.csv', index=False) # modificar

# Humano jaccard

In [23]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [24]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.42it/s]

1 188


2it [00:00,  2.40it/s]

...

58it [00:23,  2.40it/s]

58 6223


59it [00:24,  2.40it/s]

59 6313


60it [00:24,  2.42it/s]


In [25]:
df_resultado_humano_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar


df_resultado_humano_jaccard.to_csv('df_resultado_humano_jaccard_filtro_2.csv', index=False) # modificar

# Animal Jaccard

In [26]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [27]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 13.19it/s]

0 0
1 137
2 180


4it [00:00, 13.83it/s]

3 224
4 278
5 351


8it [00:00, 13.82it/s]

...

0 313
1 372


2it [00:00, 13.54it/s]

2 449
3 613


5it [00:00, 13.23it/s]

4 869


In [28]:
df_resultado_animal_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_jaccard.to_csv('df_resultado_animal_jaccard_filtro_2.csv', index=False) # modificar

# Objeto jaccard

In [29]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [30]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 12.95it/s]

0 2
1 176
2 304


4it [00:00, 13.09it/s]

3 317
4 353
5 441


8it [00:00, 12.99it/s]

...

0 378
1 404
2 451


4it [00:00, 14.49it/s]

3 465
4 806
5 963


8it [00:00, 14.30it/s]

6 1179
7 1211


In [31]:
df_resultado_objeto_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_jaccard.to_csv('df_resultado_objeto_jaccard_filtro_2.csv', index=False) # modificar

# Paisaje Jaccard

In [32]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [33]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_jaccard(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.80it/s]

1 164


2it [00:00,  3.79it/s]

...

35it [00:08,  4.08it/s]

35 3390


36it [00:09,  4.06it/s]

36 3510


37it [00:09,  4.00it/s]


In [34]:
df_resultado_paisaje_jaccard=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_jaccard.to_csv('df_resultado_paisaje_jaccard_filtro_2.csv', index=False) # modificar

# Humano Bag

In [35]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [36]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [00:00,  2.28it/s]

1 188


2it [00:00,  2.29it/s]

...

58it [00:26,  2.20it/s]

58 6223


59it [00:26,  2.19it/s]

59 6313


60it [00:27,  2.20it/s]


In [37]:
df_resultado_humano_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_bag.to_csv('df_resultado_humano_bag_filtro_2.csv', index=False) # modificar

# Animal bag

In [38]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [39]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

1it [00:00,  5.18it/s]

0 0
1 137


5it [00:00, 11.42it/s]

2 180
3 224
4 278
5 351


7it [00:00, 12.36it/s]

...

2it [00:00, 13.04it/s]

2 449
3 613


4it [00:00, 13.17it/s]

4 869


5it [00:00, 13.14it/s]


In [40]:
df_resultado_animal_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_bag.to_csv('df_resultado_animal_bag_filtro_2.csv', index=False) # modificar

# Objeto Bag

In [41]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [42]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

2it [00:00, 12.37it/s]

0 2
1 176
2 304


4it [00:00, 12.35it/s]

3 317
4 353
5 441


8it [00:00, 12.55it/s]

...

4 806
5 963


6it [00:00, 13.16it/s]

6 1179


8it [00:00, 13.06it/s]

7 1211


In [43]:
df_resultado_objeto_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_bag.to_csv('df_resultado_objeto_bag_filtro_2.csv', index=False) # modificar

# Paisaje Bag

In [44]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [45]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_bag(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [00:00,  3.40it/s]

1 164


2it [00:00,  3.41it/s]

...

35it [00:10,  3.51it/s]

35 3390


36it [00:10,  3.46it/s]

36 3510


37it [00:10,  3.48it/s]


In [46]:
df_resultado_paisaje_bag=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_bag.to_csv('df_resultado_paisaje_bag_filtro_2.csv', index=False) # modificar

# Empezamos filtro 1 metrica de semantica similarityweb

# Caso Humanos

In [112]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [115]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

62it [52:51, 51.15s/it]
67it [1:01:49, 55.36s/it]
61it [51:15, 50.41s/it]
60it [49:31, 49.53s/it]


In [116]:
df_resultado_humano_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_similarity.to_csv('df_resultado_humano_similarity.csv', index=False) # modificar

# Caso animal

In [9]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [10]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [02:16, 10.52s/it]
10it [01:15,  7.56s/it]
7it [00:35,  5.07s/it]
10it [01:15,  7.58s/it]
12it [01:52,  9.35s/it]
8it [00:47,  5.99s/it]
14it [02:33, 10.99s/it]
10it [01:15,  7.57s/it]
15it [02:55, 11.71s/it]
8it [00:47,  5.94s/it]
11it [01:32,  8.38s/it]
9it [01:00,  6.75s/it]
9it [01:00,  6.74s/it]
9it [01:00,  6.76s/it]
7it [00:35,  5.07s/it]
8it [00:47,  5.92s/it]
17it [03:47, 13.38s/it]
8it [00:47,  5.95s/it]
4it [00:10,  2.55s/it]
6it [00:25,  4.21s/it]
10it [01:15,  7.57s/it]
6it [00:25,  4.25s/it]
7it [00:35,  5.08s/it]
11it [01:32,  8.42s/it]
10it [01:15,  7.59s/it]
6it [00:25,  4.23s/it]
5it [00:17,  3.40s/it]


In [11]:
df_resultado_animal_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_similarity.to_csv('df_resultado_animal_similarity.csv', index=False) # modificar

# Caso Objeto

In [12]:
# para trabajar con los cercanos dividimos en sub dataset por etiqueta
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    # modificar dataset
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [13]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [01:44,  8.68s/it]
9it [00:55,  6.18s/it]
11it [01:24,  7.71s/it]
14it [02:19,  9.99s/it]
15it [02:41, 10.76s/it]
18it [03:55, 13.08s/it]
9it [00:55,  6.21s/it]
12it [01:42,  8.52s/it]
16it [03:05, 11.56s/it]
12it [01:41,  8.43s/it]
14it [02:20, 10.02s/it]
10it [01:09,  6.97s/it]
10it [01:09,  6.96s/it]
16it [03:03, 11.45s/it]
12it [01:41,  8.47s/it]
12it [01:41,  8.47s/it]
10it [01:09,  6.99s/it]
5it [00:15,  3.15s/it]
9it [00:55,  6.20s/it]
16it [03:03, 11.46s/it]
8it [00:43,  5.47s/it]


In [14]:
df_resultado_objeto_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_similarity.to_csv('df_resultado_objeto_similarity.csv', index=False) # modificar

# Caso paisaje

In [15]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [16]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [14:59, 25.70s/it]
52it [34:02, 39.27s/it]
39it [18:54, 29.09s/it]
39it [18:54, 29.10s/it]
48it [28:40, 35.85s/it]
37it [16:47, 27.24s/it]


In [17]:
df_resultado_paisaje_similarity=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_similarity.to_csv('df_resultado_paisaje_similarity.csv', index=False) # modificar

# Ahora websimiliraty con filtro 2

# Caso humano

In [19]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [20]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = similifuturo.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [02:48, 168.91s/it]

1 188


2it [05:38, 169.24s/it]

...

58it [2:35:47, 161.49s/it]

58 6223


59it [2:38:28, 161.37s/it]

59 6313


60it [2:41:10, 161.17s/it]


In [21]:
df_resultado_humano_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_similarity_filtro2.to_csv('df_resultado_humano_similarity_filtro2.csv', index=False) # modificar

# Caso animal

In [22]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [23]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 0


1it [00:23, 23.33s/it]

1 137


2it [00:46, 23.47s/it]

...

3it [01:15, 25.17s/it]

3 613


4it [01:40, 25.14s/it]

4 869


5it [02:05, 25.18s/it]


In [24]:
df_resultado_animal_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_similarity_filtro2.to_csv('df_resultado_animal_similarity_filtro2.csv', index=False) # modificar

# Caso objeto

In [25]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [26]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 2


1it [00:24, 24.98s/it]

1 176


2it [00:50, 25.20s/it]

...

6it [02:26, 24.45s/it]

6 1179


7it [02:51, 24.45s/it]

7 1211


8it [03:15, 24.43s/it]


In [27]:
df_resultado_objeto_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_similarity_filtro2.to_csv('df_resultado_objeto_similarity_filtro2.csv', index=False) # modificar

# Caso Paisaje

In [28]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [29]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_websimilarity(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [01:34, 94.21s/it]

1 164


2it [03:08, 94.28s/it]

...

35it [53:36, 91.89s/it]

35 3390


36it [55:08, 91.92s/it]

36 3510


37it [56:40, 91.90s/it]


In [30]:
df_resultado_paisaje_similarity_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_similarity_filtro2.to_csv('df_resultado_paisaje_similarity_filtro2.csv', index=False) # modificar

# # Empezamos filtro 1 metrica de semantica Sbert

# Caso Humano

In [32]:
%pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 2.1 MB/s eta 0:00:0031m2.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.0
    Uninstalling huggingface-hub-0.23.0:
      Successfully uninstalled huggingface-hub-0.23.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
alibi 0.9.6 requires numpy<2.0.0,>=1.16.2, but you have numpy 2.1.2 which is incompatible.

In [ ]:
%pip install "numpy<2"

In [1]:
import transformers
print(transformers.__version__)

/home/angel/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.46.1


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# funcion metrica de similitud_2 semantica
def similitud_metrica_sbert(texto_query,texto_iteracion):
    texto_query=str(texto_query)
    texto_iteracion=str(texto_iteracion)
    if texto_query!='0':
        texto_query=model.encode([texto_query])
        texto_iteracion=model.encode([texto_iteracion])
        similitud=model.similarity(texto_query,texto_iteracion)
        return float(similitud)
    else:
        return 6666

In [6]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [7]:
# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion)
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

62it [40:37, 39.31s/it]
67it [45:42, 40.94s/it]
61it [37:31, 36.91s/it]
60it [36:18, 36.31s/it]


In [8]:
df_resultado_humano_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_sbert.to_csv('df_resultado_humano_sbert.csv', index=False) # modificar

# Caso Animal

In [6]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):

    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [7]:

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [04:23, 20.30s/it]
10it [02:24, 14.48s/it]
7it [01:07,  9.70s/it]
10it [02:26, 14.67s/it]
12it [03:30, 17.56s/it]
8it [01:29, 11.15s/it]
14it [04:54, 21.02s/it]
10it [02:24, 14.41s/it]
15it [05:35, 22.34s/it]
8it [01:30, 11.26s/it]
11it [02:54, 15.84s/it]
9it [01:55, 12.89s/it]
9it [01:56, 12.94s/it]
9it [01:56, 12.91s/it]
7it [01:07,  9.67s/it]
8it [01:30, 11.26s/it]
17it [07:17, 25.76s/it]
8it [01:22, 10.28s/it]
4it [00:17,  4.48s/it]
6it [00:43,  7.26s/it]
10it [02:10, 13.04s/it]
6it [00:44,  7.42s/it]
7it [01:02,  8.87s/it]
11it [02:39, 14.53s/it]
10it [02:11, 13.15s/it]
6it [00:43,  7.31s/it]
5it [00:29,  5.98s/it]


In [8]:
df_resultado_animal_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_sbert.to_csv('df_resultado_animal_sbert.csv', index=False) # modificar

# Caso objeto

In [9]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):

    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [10]:
def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [03:07, 15.67s/it]
9it [01:35, 10.60s/it]
11it [02:34, 14.09s/it]
14it [04:22, 18.76s/it]
15it [04:37, 18.53s/it]
18it [07:23, 24.66s/it]
9it [01:38, 10.97s/it]
12it [03:03, 15.33s/it]
16it [05:30, 20.63s/it]
12it [02:55, 14.59s/it]
14it [04:10, 17.86s/it]
10it [02:04, 12.45s/it]
10it [02:00, 12.04s/it]
16it [05:18, 19.93s/it]
12it [03:00, 15.07s/it]
12it [03:00, 15.04s/it]
10it [02:02, 12.22s/it]
5it [00:26,  5.28s/it]
9it [01:39, 11.05s/it]
16it [05:24, 20.27s/it]
8it [01:16,  9.60s/it]


In [11]:
df_resultado_objeto_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_sbert.to_csv('df_resultado_objeto_sbert.csv', index=False) # modificar

# Caso paisaje

In [12]:

subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):

    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [13]:
def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # crear una lista de futuros
            futuros = []
            for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
            
            # eecoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [24:47, 42.49s/it]
52it [54:44, 63.16s/it]
39it [30:33, 47.01s/it]
39it [30:41, 47.23s/it]
48it [46:12, 57.77s/it]
37it [27:08, 44.02s/it]


In [14]:
df_resultado_paisaje_sbert=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_sbert.to_csv('df_resultado_paisaje_sbert.csv', index=False) # modificar

# Empezamos filtro 2 metrica de semantica Sbert

# Caso humano

In [15]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo[(dataset_humano_respuestas_completo['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo['singularidad']==unicos_humanos['singularidad'][i])])

In [16]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 58


1it [04:24, 264.08s/it]

1 188


2it [08:50, 265.42s/it]

...

58it [4:20:28, 270.03s/it]

58 6223


59it [4:25:02, 271.31s/it]

59 6313


60it [4:29:32, 269.54s/it]


In [17]:
df_resultado_humano_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_sbert_filtro2.to_csv('df_resultado_humano_sbert_filtro2.csv', index=False) # modificar

# Caso animal

In [18]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo[(dataset_animal_respuestas_completo['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo['singularidad']==unicos_animal['singularidad'][i])])

In [19]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 0


1it [00:41, 41.02s/it]

1 137


2it [01:20, 40.11s/it]

...

3it [02:07, 42.33s/it]

3 613


4it [02:49, 42.36s/it]

4 869


5it [03:31, 42.38s/it]


In [20]:
df_resultado_animal_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_sbert_filtro2.to_csv('df_resultado_animal_sbert_filtro2.csv', index=False) # modificar

# Caso objeto

In [21]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo[(dataset_objeto_respuestas_completo['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo['singularidad']==unicos_objetos['singularidad'][i])])

In [22]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 2


1it [00:44, 44.35s/it]

1 176


2it [01:27, 43.93s/it]

...

6it [04:10, 41.76s/it]

6 1179


7it [04:52, 41.88s/it]

7 1211


8it [05:35, 41.95s/it]


In [23]:
df_resultado_objeto_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_sbert_filtro2.to_csv('df_resultado_objeto_sbert_filtro2.csv', index=False) # modificar

# Caso Paisaje

In [24]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo[(dataset_paisaje_respuestas_completo['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo['singularidad']==unicos_paisajes['singularidad'][i])])

In [25]:
# empezamos a calcular las metricas

def comparar_textos(caso_query, resto_query, campo):
    resultados = []
    texto_query = caso_query.loc[caso_query.index.tolist()[0], campo]
    
    for x in resto_query.index.tolist():
        texto_iteracion = resto_query.loc[x, campo]
        comparacion = similitud_metrica_sbert(texto_query, texto_iteracion) # aqui la metrica de similitud
        resultados.append((int(caso_query['indice'].iloc[0]), x, campo, comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                    futuros.append(executor.submit(comparar_textos, caso_query, resto_query, campo))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

0 12


1it [02:43, 163.83s/it]

1 164


2it [05:23, 161.36s/it]

...

35it [1:29:52, 154.06s/it]

35 3390


36it [1:32:27, 154.32s/it]

36 3510


37it [1:35:01, 154.08s/it]


In [26]:
df_resultado_paisaje_sbert_filtro2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_sbert_filtro2.to_csv('df_resultado_paisaje_sbert_filtro2.csv', index=False) # modificar

# A partir de aqui seria el note 9 de metricas processiong, esto es investigacion:: ACtualizacion creo que no porque sigue la metrica con pixel to pixel

In [8]:
unicos_humanos=pd.read_csv("unicos_humanos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_animal=pd.read_csv("unicos_animal.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_objetos=pd.read_csv("unicos_objetos.csv",usecols=lambda columna: columna != 'Unnamed: 0')
unicos_paisajes=pd.read_csv("unicos_paisajes.csv",usecols=lambda columna: columna != 'Unnamed: 0')

dataset_humano_respuestas_completo=pd.read_csv("dataset_humano_respuestas_completo.csv",index_col='indice_2')
dataset_animal_respuestas_completo=pd.read_csv("dataset_animal_respuestas_completo.csv",index_col='indice_2')
dataset_objeto_respuestas_completo=pd.read_csv("dataset_objeto_respuestas_completo.csv",index_col='indice_2')
dataset_paisaje_respuestas_completo=pd.read_csv("dataset_paisaje_respuestas_completo.csv",index_col='indice_2')

In [13]:
import json

def preservar_formato(x):
    return json.dumps(x.tolist())

dataset_humano_cifar['indice_2']=dataset_humano_cifar.index
dataset_humano_cifar['data_lista']=dataset_humano_cifar[b'data'].apply(preservar_formato)

dataset_animal_cifar['indice_2']=dataset_animal_cifar.index
dataset_animal_cifar['data_lista']=dataset_animal_cifar[b'data'].apply(preservar_formato)

dataset_objeto_cifar['indice_2']=dataset_objeto_cifar.index
dataset_objeto_cifar['data_lista']=dataset_objeto_cifar[b'data'].apply(preservar_formato)

dataset_paisaje_cifar['indice_2']=dataset_paisaje_cifar.index
dataset_paisaje_cifar['data_lista']=dataset_paisaje_cifar[b'data'].apply(preservar_formato)


# TODO metricas de pixeltopixel

### La columna b'data' de los dataset ...respuestas completo parece estar rota, se recupera la info original de los bytes de la imagenes desde los dataset_cifar.... para ello la siguiente consulta

In [16]:
query_humano_recovery_image_bytes='''
SELECT dataset_humano_respuestas_completo.*, dataset_humano_cifar.data_lista FROM dataset_humano_respuestas_completo, dataset_humano_cifar
WHERE dataset_humano_respuestas_completo.indice_2=dataset_humano_cifar.indice_2
'''

query_animal_recovery_image_bytes='''
SELECT dataset_animal_respuestas_completo.*, dataset_animal_cifar.data_lista FROM dataset_animal_respuestas_completo, dataset_animal_cifar
WHERE dataset_animal_respuestas_completo.indice_2=dataset_animal_cifar.indice_2
'''

query_objeto_recovery_image_bytes='''
SELECT dataset_objeto_respuestas_completo.*, dataset_objeto_cifar.data_lista FROM dataset_objeto_respuestas_completo, dataset_objeto_cifar
WHERE dataset_objeto_respuestas_completo.indice_2=dataset_objeto_cifar.indice_2
'''

query_paisaje_recovery_image_bytes='''
SELECT dataset_paisaje_respuestas_completo.*, dataset_paisaje_cifar.data_lista FROM dataset_paisaje_respuestas_completo, dataset_paisaje_cifar
WHERE dataset_paisaje_respuestas_completo.indice_2=dataset_paisaje_cifar.indice_2
'''

dataset_humano_respuestas_completo_bytes_reparado=sql_query(query_humano_recovery_image_bytes)

dataset_animal_respuestas_completo_bytes_reparado=sql_query(query_animal_recovery_image_bytes)

dataset_objeto_respuestas_completo_bytes_reparado=sql_query(query_objeto_recovery_image_bytes)

dataset_paisaje_respuestas_completo_bytes_reparado=sql_query(query_paisaje_recovery_image_bytes)


In [17]:
def recuperar_data_array(x): # esto es necesario para que el string que contiene la imagen se interprete de forma correcta como una estructura de datos y poder acceder a los bytes
    return np.array(json.loads(x),dtype=np.uint8) 

dataset_humano_respuestas_completo_bytes_reparado['data_lista']= dataset_humano_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_animal_respuestas_completo_bytes_reparado['data_lista']= dataset_animal_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_objeto_respuestas_completo_bytes_reparado['data_lista']= dataset_objeto_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


dataset_paisaje_respuestas_completo_bytes_reparado['data_lista']= dataset_paisaje_respuestas_completo_bytes_reparado['data_lista'].apply(recuperar_data_array)


In [18]:
### resetear index
dataset_humano_respuestas_completo_bytes_reparado.set_index(dataset_humano_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_humano_respuestas_completo_bytes_reparado = dataset_humano_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [19]:
### resetear index animal
dataset_animal_respuestas_completo_bytes_reparado.set_index(dataset_animal_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_animal_respuestas_completo_bytes_reparado = dataset_animal_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [20]:
### resetear index objeto
dataset_objeto_respuestas_completo_bytes_reparado.set_index(dataset_objeto_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_objeto_respuestas_completo_bytes_reparado = dataset_objeto_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [21]:
### resetear index paisaje
dataset_paisaje_respuestas_completo_bytes_reparado.set_index(dataset_paisaje_respuestas_completo_bytes_reparado['indice_2'],inplace=True)
dataset_paisaje_respuestas_completo_bytes_reparado = dataset_paisaje_respuestas_completo_bytes_reparado.rename(columns={'indice_2': 'indice_main'})

In [22]:
from PIL import ImageChops
'''def similitud_metrica_pixel_to_pixel(image,image2):

    image=Image.fromarray(image)
    image2=Image.fromarray(image2)

    diferencia = ImageChops.difference(image, image2)
    promedio_diferencia = np.mean(np.abs(np.array(diferencia)))

    return promedio_diferencia
    # No existe el 6666 ya que no hay preguntas


CREO QUE LA FUNCION SERIA:

'''
def similitud_metrica_pixel_to_pixel(image,image2):

    image=Image.fromarray(image.reshape(3,32,32).transpose(1,2,0))
    image2=Image.fromarray(image2.reshape(3,32,32).transpose(1,2,0))

    diferencia = ImageChops.difference(image, image2)
    promedio_diferencia = np.mean(np.abs(np.array(diferencia)))

    return promedio_diferencia


pARECE QUE HACE FALTA TRANSPORNER LA IMAGEN

# Filtro 1

## No eliminar para comentar despues

In [253]:
image = dataset_humano_cifar[b'data'][58]
image = image.reshape(3, 32, 32).transpose(1, 2, 0)

image2=dataset_humano_cifar[b'data'][65]
image2= image2.reshape(3,32,32).transpose(1,2,0)

prueba=np.mean(abs(image2-image))
print(prueba)
#image=Image.fromarray(image)
#image2=Image.fromarray(image2)

#pixel_to_pixel=ImageChops.difference(image,image2)



138.83951822916666


# Caso humano

In [23]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_humanos)):
    #AQUI TAMBIEN AÑADIR LOS OTROS DATASETS
    subdataset_para_comparativa_filtro_1.append(dataset_humano_respuestas_completo_bytes_reparado[(dataset_humano_respuestas_completo_bytes_reparado['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo_bytes_reparado['singularidad']==unicos_humanos['singularidad'][i])])

In [24]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

0it [00:00, ?it/s]

62it [00:00, 98.39it/s] 
67it [00:00, 93.29it/s]
61it [00:00, 97.91it/s]
60it [00:00, 101.16it/s]


In [25]:
df_resultado_humano_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_pixel_to_pixel.to_csv('df_resultado_humano_pixel_to_pixel_v2.csv', index=False) # modificar

# Caso animal

In [26]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_animal)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_animal_respuestas_completo_bytes_reparado[(dataset_animal_respuestas_completo_bytes_reparado['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo_bytes_reparado['singularidad']==unicos_animal['singularidad'][i])])

In [27]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

13it [00:00, 289.01it/s]
10it [00:00, 372.22it/s]
7it [00:00, 506.17it/s]
10it [00:00, 357.68it/s]
12it [00:00, 349.95it/s]
8it [00:00, 437.72it/s]
14it [00:00, 307.42it/s]
10it [00:00, 377.96it/s]
15it [00:00, 299.41it/s]
8it [00:00, 392.80it/s]
11it [00:00, 360.36it/s]
9it [00:00, 392.02it/s]
9it [00:00, 422.72it/s]
9it [00:00, 371.21it/s]
7it [00:00, 441.79it/s]
8it [00:00, 448.87it/s]
17it [00:00, 276.59it/s]
8it [00:00, 443.28it/s]
4it [00:00, 606.14it/s]
6it [00:00, 459.83it/s]
10it [00:00, 401.45it/s]
6it [00:00, 450.08it/s]
7it [00:00, 453.87it/s]
11it [00:00, 369.12it/s]
10it [00:00, 370.34it/s]
6it [00:00, 486.63it/s]
5it [00:00, 604.54it/s]


In [28]:
df_resultado_animal_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_pixel_to_pixel.to_csv('df_resultado_animal_pixel_to_pixel_v2.csv', index=False) # modificar

# Caso objeto

In [29]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_objeto_respuestas_completo_bytes_reparado[(dataset_objeto_respuestas_completo_bytes_reparado['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo_bytes_reparado['singularidad']==unicos_objetos['singularidad'][i])])

In [30]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

12it [00:00, 302.74it/s]
9it [00:00, 381.28it/s]
11it [00:00, 358.10it/s]
14it [00:00, 304.52it/s]
15it [00:00, 310.46it/s]
18it [00:00, 252.54it/s]
9it [00:00, 395.23it/s]
12it [00:00, 380.88it/s]
16it [00:00, 283.66it/s]
12it [00:00, 346.31it/s]
14it [00:00, 298.29it/s]
10it [00:00, 392.03it/s]
10it [00:00, 406.67it/s]
16it [00:00, 280.45it/s]
12it [00:00, 341.72it/s]
12it [00:00, 315.85it/s]
10it [00:00, 368.26it/s]
5it [00:00, 497.98it/s]
9it [00:00, 408.08it/s]
16it [00:00, 258.40it/s]
8it [00:00, 475.50it/s]


In [31]:
df_resultado_objeto_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_pixel_to_pixel.to_csv('df_resultado_objeto_pixel_to_pixel_v2.csv', index=False) # modificar

# Caso papisaje

In [32]:
subdataset_para_comparativa_filtro_1 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_1.append(dataset_paisaje_respuestas_completo_bytes_reparado[(dataset_paisaje_respuestas_completo_bytes_reparado['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo_bytes_reparado['singularidad']==unicos_paisajes['singularidad'][i])])

In [33]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Suponiendo que similitud_metrica_levestehin y subdataset_para_comparativa_filtro_1 están definidos

def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados

lista_comparacion = []

with ThreadPoolExecutor() as executor:
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_1)):
        indice = subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].index.tolist()
        
        for i, p in tqdm(enumerate(indice)):
            caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[i], :]).T
            resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_1[indice_subdataset_para_comparativa].loc[indice[:i] + indice[i + 1:], :])
            
            # Crear una lista de futuros
            futuros = []
            #for campo in caso_query.columns[4:-1].tolist():
                # Enviar la tarea al executor
            futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
            
            # Recoger los resultados a medida que se completan
            for futuro in as_completed(futuros):
                resultados = futuro.result()
                lista_comparacion.extend(resultados)

35it [00:00, 160.00it/s]
52it [00:00, 113.51it/s]
39it [00:00, 150.81it/s]
39it [00:00, 150.18it/s]
48it [00:00, 121.48it/s]
37it [00:00, 148.89it/s]


In [34]:
df_resultado_paisaje_pixel_to_pixel=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_pixel_to_pixel.to_csv('df_resultado_paisaje_pixel_to_pixel_v2.csv', index=False) # modificar

# Filtro 2 pixel to pixel

# Caso Humano

In [35]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_humanos)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_humano_respuestas_completo_bytes_reparado[(dataset_humano_respuestas_completo_bytes_reparado['etiqueta']==unicos_humanos['etiqueta'][i]) &
                                   (dataset_humano_respuestas_completo_bytes_reparado['singularidad']==unicos_humanos['singularidad'][i])])

In [36]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

4it [00:00, 34.85it/s]

0 58
1 188
2 323
3 490
4 578
5 598
6 637
7 657


12it [00:00, 35.75it/s]

8 692
9 707
10 740
11 767
12 888
13 929
14 968
15 1021


20it [00:00, 36.03it/s]

...

52it [00:01, 35.48it/s]

52 5480
53 5487
54 5661
55 5802


56it [00:01, 35.57it/s]

56 5853
57 5921
58 6223
59 6313


60it [00:01, 35.02it/s]


In [37]:
df_resultado_humano_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_humano_pixel_to_pixel_filtro_2.to_csv('df_resultado_humano_pixel_to_pixel_filtro_2_v2.csv', index=False) # modificar

# Caso animal

In [38]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_animal)):
    
    #aqui seria campiar por el resto de etiquetas, pero por el resto de dataframes tambien?
    subdataset_para_comparativa_filtro_2.append(dataset_animal_respuestas_completo_bytes_reparado[(dataset_animal_respuestas_completo_bytes_reparado['etiqueta']==unicos_animal['etiqueta'][i]) &
                                   (dataset_animal_respuestas_completo_bytes_reparado['singularidad']==unicos_animal['singularidad'][i])])

In [39]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

13it [00:00, 182.62it/s]


0 0
1 137
2 180
3 224
4 278
5 351
6 430
7 529
8 683
9 686
10 762
11 818
12 870


10it [00:00, 173.99it/s]


0 1
1 155
2 238
3 343
4 453
5 677
6 711
7 793
8 906
9 914


7it [00:00, 167.14it/s]


...

10it [00:00, 176.03it/s]
0it [00:00, ?it/s]

0 177
1 272
2 399
3 470
4 623
5 661


6it [00:00, 145.40it/s]
0it [00:00, ?it/s]

0 313
1 372
2 449
3 613
4 869


5it [00:00, 165.72it/s]


In [40]:
df_resultado_animal_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_animal_pixel_to_pixel_filtro_2.to_csv('df_resultado_animal_pixel_to_pixel_filtro_2_v2.csv', index=False) # modificar

# Caso objeto

In [41]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_objetos)):
    
    subdataset_para_comparativa_filtro_2.append(dataset_objeto_respuestas_completo_bytes_reparado[(dataset_objeto_respuestas_completo_bytes_reparado['etiqueta']==unicos_objetos['etiqueta'][i]) &
                                   (dataset_objeto_respuestas_completo_bytes_reparado['singularidad']==unicos_objetos['singularidad'][i])])

In [42]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

12it [00:00, 150.04it/s]


0 2
1 176
2 304
3 317
4 353
5 441
6 459
7 587
8 658
9 789
10 822
11 915


9it [00:00, 138.05it/s]


0 5
1 198
2 222
3 263
4 496
5 668
6 953
7 1083
8 1186


0it [00:00, ?it/s]

...

16it [00:00, 147.52it/s]


15 1213


0it [00:00, ?it/s]

0 378
1 404
2 451
3 465
4 806
5 963
6 1179
7 1211


8it [00:00, 141.70it/s]


In [43]:
df_resultado_objeto_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_objeto_pixel_to_pixel_filtro_2.to_csv('df_resultado_objeto_pixel_to_pixel_filtro_2_v2.csv', index=False) # modificar


# Caso paisaje

In [44]:
subdataset_para_comparativa_filtro_2 = []
for i in range(0,len(unicos_paisajes)):
    
    subdataset_para_comparativa_filtro_2.append(dataset_paisaje_respuestas_completo_bytes_reparado[(dataset_paisaje_respuestas_completo_bytes_reparado['etiqueta']==unicos_paisajes['etiqueta'][i]) &
                                   (dataset_paisaje_respuestas_completo_bytes_reparado['singularidad']==unicos_paisajes['singularidad'][i])])

In [45]:
# empezamos a calcular las metricas
def comparar_imagenes(caso_query, resto_query):
    resultados = []
    imagen_query = caso_query.loc[caso_query.index.tolist()[0], 'data_lista']
    
    for x in resto_query.index.tolist():
        imagen_iteracion = resto_query.loc[x, 'data_lista']
        comparacion = similitud_metrica_pixel_to_pixel(imagen_query, imagen_iteracion)
        resultados.append((int(caso_query['indice_main'].iloc[0]), x, 'data_lista', comparacion))
    
    return resultados


lista_comparacion = []

with ThreadPoolExecutor() as executor:    
    for indice_subdataset_para_comparativa in range(len(subdataset_para_comparativa_filtro_2)):

            indice = subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].index.tolist()

            lista_para_concat_actual=subdataset_para_comparativa_filtro_2.copy()
            lista_para_concat_actual.pop(indice_subdataset_para_comparativa)
            indice_resto_query_otrosub_dataset=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]]).index.tolist()
            
            for i, p in tqdm(enumerate(indice)):
                print(i,p)
                caso_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa].loc[indice[i], :]).T
                #resto_query = pd.DataFrame(subdataset_para_comparativa_filtro_2[indice_subdataset_para_comparativa if indice_subdataset_para_comparativa==3 else indice_subdataset_para_comparativa+1])
                resto_query=pd.concat([lista_para_concat_actual[0],lista_para_concat_actual[1],lista_para_concat_actual[2]])
            
                # crear una lista de futuros
                futuros = []
                #for campo in caso_query.columns[4:-1].tolist():
                    # Enviar la tarea al executor
                futuros.append(executor.submit(comparar_imagenes, caso_query, resto_query))
                
                # eecoger los resultados a medida que se completan
                for futuro in as_completed(futuros):
                    resultados = futuro.result()
                    lista_comparacion.extend(resultados)

6it [00:00, 51.08it/s]

0 12
1 164
2 274
3 314
4 318
5 324
6 362
7 466
8 472
9 625
10 871


18it [00:00, 50.86it/s]

11 1029
12 1136
13 1143
14 1296
15 1402
16 1660
17 1854
18 1913
19 1919
20 2147
21 2542


30it [00:00, 50.68it/s]

...

24 1888
25 2242
26 2430
27 2431
28 2625


30it [00:00, 54.21it/s]

29 2642
30 2701
31 2755
32 2883
33 3126
34 3292
35 3390


37it [00:00, 53.58it/s]

36 3510


In [46]:
df_resultado_paisaje_pixel_to_pixel_filtro_2=pd.DataFrame(data=lista_comparacion, columns=['par_1', 'par_2', 'pregunta', 'valor_comparacion']) # modificar

df_resultado_paisaje_pixel_to_pixel_filtro_2.to_csv('df_resultado_paisaje_pixel_to_pixel_filtro_2_v2.csv', index=False) # modificar
